In [ ]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
    # Загрузка модели и токенизатора mBART
    tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
    model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

    def compress_text_mbart(text, max_length=150, min_length=50, length_penalty=2.0):
        """
        Функция для сжатия текста с использованием модели mBART.
        Можно настроить длину сжатого текста через параметры max_length, min_length и length_penalty.
    """
    # Указываем исходный язык
    tokenizer.src_lang = "ru_RU"
    
    # Токенизация входного текста
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    
    # Указываем целевой язык
    target_lang = "ru_RU"
    forced_bos_token_id = tokenizer.lang_code_to_id[target_lang]
    
    # Генерация сжатого текста с указанием минимальной и максимальной длины, а также штрафа за длину
    summary_ids = model.generate(
        inputs["input_ids"], 
        max_length=max_length,          # Максимальная длина сжатого текста
        min_length=min_length,          # Минимальная длина сжатого текста
        length_penalty=length_penalty,  # Штраф за длину (значения ниже 1 -> короче, выше 1 -> длиннее)
        num_beams=4,                    # Количество лучей для генерации (beam search)
        early_stopping=True,            # Остановка при достижении условий
        forced_bos_token_id=forced_bos_token_id
    )
    
    
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    return summary

# Пример использования
text = """
Несколько лет тому назад в одном из своих поместий жил старинный русский барин, Кирила Петрович Троекуров. Его богатство, знатный род и связи давали ему большой вес в губерниях, где находилось его имение. Соседи рады были угождать малейшим его прихотям; губернские чиновники трепетали при его имени; Кирила Петрович принимал знаки подобострастия как надлежащую дань; дом его всегда был полон гостями, готовыми тешить его барскую праздность, разделяя шумные, а иногда и буйные его увеселения. Никто не дерзал отказываться от его приглашения или в известные дни не являться с должным почтением в село Покровское. В домашнем быту Кирила Петрович выказывал все пороки человека необразованного. Избалованный всем, что только окружало его, он привык давать полную волю всем порывам пылкого своего нрава и всем затеям довольно ограниченного ума. Несмотря на необыкновенную силу физических способностей, он раза два в неделю страдал от обжорства и каждый вечер бывал навеселе. В одном из флигелей его дома жили шестнадцать горничных, занимаясь рукоделиями, свойственными их полу. Окны во флигеле были загорожены деревянною решеткою; двери запирались замками, от коих ключи хранились у Кирила Петровича. Молодые затворницы в положенные часы сходили в сад и прогуливались под надзором двух старух. От времени до времени Кирила Петрович выдавал некоторых из них замуж, и новые поступали на их место. С крестьянами и дворовыми обходился он строго и своенравно; несмотря на то, они были ему преданы: они тщеславились богатством и славою своего господина и в свою очередь позволяли себе многое в отношении к их соседям, надеясь на его сильное покровительство.
"""
compressed_text = compress_text_mbart(text, max_length=100, min_length=50, length_penalty=1.5)
print("Сжатый текст:", compressed_text)

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer

# Загружаем модель и токенизатор для русского языка
model = MBartForConditionalGeneration.from_pretrained('facebook/mbart-large-50-many-to-many-mmt')
tokenizer = MBart50Tokenizer.from_pretrained('facebook/mbart-large-50-many-to-many-mmt')

# Указываем целевой язык для генерации - русский (ru_RU)
tokenizer.src_lang = "ru_RU"

# Ваш текст книги или главы для суммаризации

# Токенизация текста
inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)

# Генерация суммаризации
summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

# Расшифровка результата
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)


In [ ]:
!pip install -r requirements.txt

In [ ]:
pip install torch

In [ ]:
!pip install -e .
!pip install -U mamba-ssm causal-conv1d

In [ ]:
!git clone https://github.com/Zyphra/Zamba2.git
%cd Zamba2
!pip install -e .
!pip install -U mamba-ssm causal-conv1d



In [ ]:
!pip install qwen

In [ ]:
from mamba_model import MambaModel
from mamba_config import MambaConfig
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Zyphra/Zamba2-2.7B")
input_text = 'A funny prompt would be '
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")["input_ids"].transpose(0,1)
model = MambaModel.from_pretrained(model_name = "Zyphra/Zamba2-2.7B").cuda().half()
tokens_to_generate = 20
model.eval()
with torch.no_grad():
    for _ in range(tokens_to_generate):
        out = model(input_ids)
        out_last = out[:, -1]
        idx = torch.argmax(out_last)[None, None]
        input_ids = torch.cat((input_ids, idx), dim=0)
input_ids = input_ids.transpose(0, 1)[0]
print(repr(tokenizer.decode(input_ids.cpu().numpy().tolist())))

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# torch.set_default_device("auto")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

inputs = tokenizer(akp, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=166445)
text = tokenizer.batch_decode(outputs)[0]
print(text)


In [ ]:
!nvidia-smi


In [ ]:
kill -9 <process_id>


In [ ]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:<enter-size-here>"


In [ ]:
torch.cuda.empty_cache()
gc.collect()


In [ ]:
inputs = tokenizer(akp, return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=166444)
text = tokenizer.batch_decode(outputs)[0]
print(text)


In [ ]:
prompt = pipe.tokenizer.apply_chat_template(akp, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Загрузка модели и токенизатора T5 для русского языка
tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-base-multitask")
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-base-multitask")

def compress_text_t5(text, max_length=150, min_length=50, length_penalty=2.0):
    """
    Функция для сжатия текста с использованием модели T5.
    Параметры:
    - max_length: максимальная длина сжатого текста в токенах
    - min_length: минимальная длина сжатого текста в токенах
    - length_penalty: штраф за длину (значения ниже 1 создают более короткие тексты, выше 1 — более длинные)
    """
    # Префикс для задачи сжатия
    task_prefix = "summarize: "
    
    # Токенизация входного текста
    inputs = tokenizer(task_prefix + text, return_tensors="pt", max_length=1024, truncation=True)
    
    # Генерация сжатого текста с указанием минимальной и максимальной длины, а также штрафа за длину
    summary_ids = model.generate(
        inputs["input_ids"], 
        max_length=max_length,          # Максимальная длина сжатого текста
        min_length=min_length,          # Минимальная длина сжатого текста
        length_penalty=length_penalty,  # Штраф за длину
        num_beams=4,                    # Количество лучей для генерации
        early_stopping=True             # Остановка генерации при достижении условий
    )
    
    # Раскодирование результата
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    return summary

# Пример использования
# text = """
# Несколько лет тому назад в одном из своих поместий жил старинный русский барин, Кирила Петрович Троекуров. Его богатство, знатный род и связи давали ему большой вес в губерниях, где находилось его имение. Соседи рады были угождать малейшим его прихотям; губернские чиновники трепетали при его имени; Кирила Петрович принимал знаки подобострастия как надлежащую дань; дом его всегда был полон гостями, готовыми тешить его барскую праздность, разделяя шумные, а иногда и буйные его увеселения. Никто не дерзал отказываться от его приглашения или в известные дни не являться с должным почтением в село Покровское. В домашнем быту Кирила Петрович выказывал все пороки человека необразованного. Избалованный всем, что только окружало его, он привык давать полную волю всем порывам пылкого своего нрава и всем затеям довольно ограниченного ума. Несмотря на необыкновенную силу физических способностей, он раза два в неделю страдал от обжорства и каждый вечер бывал навеселе. В одном из флигелей его дома жили шестнадцать горничных, занимаясь рукоделиями, свойственными их полу. Окны во флигеле были загорожены деревянною решеткою; двери запирались замками, от коих ключи хранились у Кирила Петровича. Молодые затворницы в положенные часы сходили в сад и прогуливались под надзором двух старух. От времени до времени Кирила Петрович выдавал некоторых из них замуж, и новые поступали на их место. С крестьянами и дворовыми обходился он строго и своенравно; несмотря на то, они были ему преданы: они тщеславились богатством и славою своего господина и в свою очередь позволяли себе многое в отношении к их соседям, надеясь на его сильное покровительство.
# """
# compressed_text = compress_text_t5(text, max_length=100, min_length=50, length_penalty=1.5)
# print("Сжатый текст:", compressed_text)


In [ ]:
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

# Загрузка BigBird-Pegasus модели и токенизатора
tokenizer = AutoTokenizer.from_pretrained('google/bigbird-pegasus-large-arxiv')
model = BigBirdPegasusForConditionalGeneration.from_pretrained('google/bigbird-pegasus-large-arxiv')


# Токенизация текста с поддержкой длинного контекста (до 4096 токенов)
inputs = tokenizer(text, return_tensors='pt', max_length=4096, truncation=True)

# Генерация суммаризации
summary_ids = model.generate(inputs['input_ids'], max_length=200, num_beams=5, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)


In [ ]:
summary

In [ ]:
from transformers import LongformerTokenizer, LongformerForSeq2SeqLM

# Загрузка модели Longformer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerForSeq2SeqLM.from_pretrained('allenai/longformer-base-4096')

# Пример текста

# Токенизация текста с длиной до 4096 токенов
input_ids = tokenizer.encode(text, return_tensors="pt", max_length=4096, truncation=True)

# Генерация суммаризации
summary_ids = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)


In [ ]:
len("""Несколько лет назад в одном из своих поместий жил старинный русский барин, Кирила Петрович Троекуров. Его богатство, знатный род и связи давали ему большой вес в губерниях, где находилось его имение. Его богатство, знатный род и связи давали ему большой вес в губерниях, где находилось его имение. Соседи рады были угождать малейшим его прихотям""")

In [ ]:
compressed_text_long = compress_text_t5(text, max_length=1000, min_length=100, length_penalty=100.0)
print("Развернутое сжатие:", compressed_text_long)


In [ ]:
len(compressed_text_long)

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Пример текста

inputs = tokenizer(text, return_tensors='pt', max_length=1024, truncation=True)
summary_ids = model.generate(inputs['input_ids'], max_length=1500, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Загрузка модели и токенизатора GPT
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

In [ ]:

def generate_question(context, max_new_tokens=50):
    """
    Функция для генерации вопроса на основе контекста.
    """
    prompt = f"Generate a question based on the following text: {context}"
    inputs = tokenizer.encode(prompt, return_tensors='pt', max_length=512, truncation=True)
    outputs = model.generate(inputs, max_new_tokens=max_new_tokens, num_beams=4, early_stopping=True)
    question = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

# Пример использования
context = """
Главный герой отправился в путешествие, где встретил множество препятствий и приключений. Одним из ключевых событий стал конфликт с местным населением.
"""
question = generate_question(context)
print("Сгенерированный вопрос:", question)


In [ ]:
question

In [ ]:
!wget https://royallib.com/get/txt/gorkiy_maksim/na_dne.zip

In [ ]:
!unzip na_dne.zip


In [ ]:
import os

In [ ]:
with open(os.listdir()[-1], 'r', encoding='utf-8') as file:
    file.readline()

In [ ]:
!sudo apt -Y install enca

In [ ]:
!cat 

In [ ]:
!huggingface-cli login 

In [ ]:
!kaggle config view

In [ ]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B"

pipe = pipeline(
    "text-generation", 
    model=model_id, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

pipe(")


In [ ]:
%%capture

!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
# One must patch the DPO Trainer first!
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/zephyr-sft-bnb-4bit", # Choose ANY! eg mistralai/Mistral-7B-Instruct-v0.2
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model

In [ ]:
model.eval()
FastLanguageModel.for_inference(model)


In [ ]:
a=int(len(akp)/2.5)

In [ ]:
acp=akp[:a]

In [ ]:
len(akp)

In [ ]:
inputs = tokenizer([acp], return_tensors="pt").to("cuda")
tokenized_output = model.generate(**inputs, use_cache=True)
decoded_output = tokenizer.batch_decode(tokenized_output, skip_special_tokens=True)



In [ ]:
akp = """Твоя задача вывести на русском выжимку данного текста чтобы его было можно прочитать за данное количество дней, если каждый день человек читает по 2 часа, а в час он прочитывает 9000 слов, если что его точно можно прочитать за данное количество дней 
количество дней=2, текст: """ + text

In [ ]:
text = """Глава первая

И жить торопится, и чувствовать спешит.
Князь Вяземский 1



I


«Мой дядя самых честных правил,
Когда не в шутку занемог,
Он уважать себя заставил
И лучше выдумать не мог.
Его пример другим наука;
Но, боже мой, какая скука
С больным сидеть и день и ночь,
Не отходя ни шагу прочь!
Какое низкое коварство
Полуживого забавлять,
Ему подушки поправлять,
Печально подносить лекарство,
Вздыхать и думать про себя:
Когда же черт возьмет тебя!»


II


Так думал молодой повеса,
Летя в пыли на почтовых,
Всевышней волею Зевеса
Наследник всех своих родных. —
Друзья Людмилы и Руслана!
С героем моего романа
Без предисловий, сей же час
Позвольте познакомить вас:
Онегин, добрый мой приятель,
Родился на брегах Невы,
Где, может быть, родились вы
Или блистали, мой читатель;
Там некогда гулял и я:
Но вреден север для меня2.


III


Служив отлично-благородно,
Долгами жил его отец,
Давал три бала ежегодно
И промотался наконец.
Судьба Евгения хранила:
Сперва Madame за ним ходила,
Потом Monsieur ее сменил;
Ребенок был резов, но мил.
Monsieur l’Abbe€, француз убогой,
Чтоб не измучилось дитя,
Учил его всему шутя,
Не докучал моралью строгой,
Слегка за шалости бранил
И в Летний сад гулять водил.


IV


Когда же юности мятежной
Пришла Евгению пора,
Пора надежд и грусти нежной,
Monsieur прогнали со двора.
Вот мой Онегин на свободе;
Острижен по последней моде;
Как dandy 3 лондонский одет —
И наконец увидел свет.
Он по-французски совершенно
Мог изъясняться и писал;
Легко мазурку танцевал
И кланялся непринужденно;
Чего ж вам больше? Свет решил,
Что он умен и очень мил.


V


Мы все учились понемногу
Чему-нибудь и как-нибудь,
Так воспитаньем, слава богу,
У нас немудрено блеснуть.
Онегин был, по мненью многих
(Судей решительных и строгих),
Ученый малый, но педант4.
Имел он счастливый талант
Без принужденья в разговоре
Коснуться до всего слегка,
С ученым видом знатока
Хранить молчанье в важном споре
И возбуждать улыбку дам
Огнем нежданных эпиграмм.


VI


Латынь из моды вышла ныне:
Так, если правду вам сказать,
Он знал довольно по-латыни,
Чтоб эпиграфы разбирать,
Потолковать об Ювенале,
В конце письма поставить vale 5, 
Да помнил, хоть не без греха,
Из Энеиды два стиха.
Он рыться не имел охоты
В хронологической пыли
Бытописания земли;
Но дней минувших анекдоты,
От Ромула до наших дней,
Хранил он в памяти своей.


VII


Высокой страсти не имея
Для звуков жизни не щадить,
Не мог он ямба от хорея,
Как мы ни бились, отличить.
Бранил Гомера, Феокрита;
Зато читал Адама Смита
И был глубокий эконом,
То есть умел судить о том,
Как государство богатеет,
И чем живет, и почему
Не нужно золота ему,
Когда простой продукт имеет.
Отец понять его не мог
И земли отдавал в залог.


VIII


Всего, что знал еще Евгений,
Пересказать мне недосуг;
Но в чем он истинный был гений,
Что знал он тверже всех наук,
Что было для него измлада
И труд, и мука, и отрада,
Что занимало целый день
Его тоскующую лень, —
Была наука страсти нежной,
Которую воспел Назон,
За что страдальцем кончил он
Свой век блестящий и мятежный
В Молдавии, в глуши степей,
Вдали Италии своей.


IX


……………………………………
……………………………………
……………………………………


X


Как рано мог он лицемерить,
Таить надежду, ревновать,
Разуверять, заставить верить,
Казаться мрачным, изнывать,
Являться гордым и послушным,
Внимательным иль равнодушным!
Как томно был он молчалив,
Как пламенно красноречив,
В сердечных письмах как небрежен!
Одним дыша, одно любя,
Как он умел забыть себя!
Как взор его был быстр и нежен,
Стыдлив и дерзок, а порой
Блистал послушною слезой!


XI


Как он умел казаться новым,
Шутя невинность изумлять,
Пугать отчаяньем готовым,
Приятной лестью забавлять,
Ловить минуту умиленья,
Невинных лет предубежденья
Умом и страстью побеждать,
Невольной ласки ожидать,
Молить и требовать признанья,
Подслушать сердца первый звук,
Преследовать любовь и вдруг
Добиться тайного свиданья…
И после ей наедине
Давать уроки в тишине!


XII


Как рано мог уж он тревожить
Сердца кокеток записных!
Когда ж хотелось уничтожить
Ему соперников своих,
Как он язвительно злословил!
Какие сети им готовил!
Но вы, блаженные мужья,
С ним оставались вы друзья:
Его ласкал супруг лукавый,
Фобласа давний ученик,
И недоверчивый старик,
И рогоносец величавый,
Всегда довольный сам собой,
Своим обедом и женой.


XIII. XIV


……………………………………
……………………………………
……………………………………


XV


Бывало, он еще в постеле:
К нему записочки несут.
Что? Приглашенья? В самом деле,
Три дома на вечер зовут:
Там будет бал, там детский праздник.
Куда ж поскачет мой проказник?
С кого начнет он? Всё равно:
Везде поспеть немудрено.
Покамест в утреннем уборе,
Надев широкий боливар 6, 
Онегин едет на бульвар,
И там гуляет на просторе,
Пока недремлющий брегет
Не прозвонит ему обед.


XVI


Уж темно: в санки он садится.
«Пади, пади!» – раздался крик;
Морозной пылью серебрится
Его бобровый воротник.
К Talon 7помчался: он уверен,
Что там уж ждет его Каверин.
Вошел: и пробка в потолок,
Вина кометы брызнул ток;
Пред ним roast-beef 8окровавленный
И трюфли, роскошь юных лет,
Французской кухни лучший цвет,
И Страсбурга пирог нетленный
Меж сыром лимбургским живым
И ананасом золотым.


XVII


Еще бокалов жажда просит
Залить горячий жир котлет,
Но звон брегета им доносит,
Что новый начался балет.
Театра злой законодатель,
Непостоянный обожатель
Очаровательных актрис,
Почетный гражданин кулис,
Онегин полетел к театру,
Где каждый, вольностью дыша,
Готов охлопать entrechat 9, 
Обшикать Федру, Клеопатру,
Моину вызвать (для того,
Чтоб только слышали его).


XVIII


Волшебный край! там в стары годы,
Сатиры смелый властелин,
Блистал Фонвизин, друг свободы,
И переимчивый Княжнин;
Там Озеров невольны дани
Народных слез, рукоплесканий
С младой Семеновой делил;
Там наш Катенин воскресил
Корнеля гений величавый;
Там вывел колкий Шаховской
Своих комедий шумный рой,
Там и Дидло10венчался славой,
Там, там под сению кулис
Младые дни мои неслись.


XIX


Мои богини! что вы? где вы?
Внемлите мой печальный глас:
Всё те же ль вы? другие ль девы,
Сменив, не заменили вас?
Услышу ль вновь я ваши хоры?
Узрю ли русской Терпсихоры
Душой исполненный полет?
Иль взор унылый не найдет
Знакомых лиц на сцене скучной,
И, устремив на чуждый свет
Разочарованный лорнет,
Веселья зритель равнодушный,
Безмолвно буду я зевать
И о былом воспоминать?


XX


Театр уж полон; ложи блещут;
Партер и кресла, всё кипит;
В райке нетерпеливо плещут,
И, взвившись, занавес шумит.
Блистательна, полувоздушна,
Смычку волшебному послушна,
Толпою нимф окружена,
Стоит Истомина; она,
Одной ногой касаясь пола,
Другою медленно кружит,
И вдруг прыжок, и вдруг летит,
Летит, как пух от уст Эола;
То стан совьет, то разовьет,
И быстрой ножкой ножку бьет.


XXI


Всё хлопает. Онегин входит,
Идет меж кресел по ногам,
Двойной лорнет скосясь наводит
На ложи незнакомых дам;
Все ярусы окинул взором,
Всё видел: лицами, убором
Ужасно недоволен он;
С мужчинами со всех сторон
Раскланялся, потом на сцену
В большом рассеянье взглянул,
Отворотился – и зевнул,
И молвил: «Всех пора на смену;
Балеты долго я терпел,
Но и Дидло5) мне надоел».


XXII


Еще амуры, черти, змеи
На сцене скачут и шумят;
Еще усталые лакеи
На шубах у подъезда спят;
Еще не перестали топать,
Сморкаться, кашлять, шикать, хлопать;
Еще снаружи и внутри
Везде блистают фонари;
Еще, прозябнув, бьются кони,
Наскуча упряжью своей,
И кучера, вокруг огней,
Бранят господ и бьют в ладони:
А уж Онегин вышел вон;
Домой одеться едет он.


XXIII


Изображу ль в картине верной
Уединенный кабинет,
Где мод воспитанник примерный
Одет, раздет и вновь одет?
Всё, чем для прихоти обильной
Торгует Лондон щепетильный
И по Балтическим волнам
За лес и сало возит нам,
Всё, что в Париже вкус голодный,
Полезный промысел избрав,
Изобретает для забав,
Для роскоши, для неги модной, —
Всё украшало кабинет
Философа в осьмнадцать лет.


XXIV


Янтарь на трубках Цареграда,
Фарфор и бронза на столе,
И, чувств изнеженных отрада,
Духи в граненом хрустале;
Гребенки, пилочки стальные,
Прямые ножницы, кривые,
И щетки тридцати родов
И для ногтей, и для зубов.
Руссо (замечу мимоходом)
Не мог понять, как важный Грим
Смел чистить ногти перед ним,
Красноречивым сумасбродом11.
Защитник вольности и прав
В сем случае совсем неправ.


XXV


Быть можно дельным человеком
И думать о красе ногтей:
К чему бесплодно спорить с веком?
Обычай деспот меж людей.
Второй Чадаев, мой Евгений,
Боясь ревнивых осуждений,
В своей одежде был педант
И то, что мы назвали франт.
Он три часа по крайней мере
Пред зеркалами проводил
И из уборной выходил
Подобный ветреной Венере,
Когда, надев мужской наряд,
Богиня едет в маскарад.


XXVI


В последнем вкусе туалетом
Заняв ваш любопытный взгляд,
Я мог бы пред ученым светом
Здесь описать его наряд;
Конечно б, это было смело,
Описывать мое же дело:
Но панталоны, фрак, жилет, 
Всех этих слов на русском нет;
А вижу я, винюсь пред вами,
Что уж и так мой бедный слог
Пестреть гораздо б меньше мог
Иноплеменными словами,
Хоть и заглядывал я встарь
В Академический Словарь.


XXVII


У нас теперь не то в предмете:
Мы лучше поспешим на бал,
Куда стремглав в ямской карете
Уж мой Онегин поскакал.
Перед померкшими домами
Вдоль сонной улицы рядами
Двойные фонари карет
Веселый изливают свет
И радуги на снег наводят;
Усеян плошками кругом,
Блестит великолепный дом;
По цельным окнам тени ходят,
Мелькают профили голов
И дам и модных чудаков.


XXVIII


Вот наш герой подъехал к сеням;
Швейцара мимо он стрелой
Взлетел по мраморным ступеням,
Расправил волоса рукой,
Вошел. Полна народу зала;
Музыка уж греметь устала;
Толпа мазуркой занята;
Кругом и шум и теснота;
Бренчат кавалергарда шпоры;
Летают ножки милых дам;
По их пленительным следам
Летают пламенные взоры,
И ревом скрыпок заглушен
Ревнивый шепот модных жен.


XXIX


Во дни веселий и желаний
Я был от балов без ума:
Верней нет места для признаний
И для вручения письма.
О вы, почтенные супруги!
Вам предложу свои услуги;
Прошу мою заметить речь:
Я вас хочу предостеречь.
Вы также, маменьки, построже
За дочерьми смотрите вслед:
Держите прямо свой лорнет!
Не то… не то, избави Боже!
Я это потому пишу,
Что уж давно я не грешу.


XXX


Увы, на разные забавы
Я много жизни погубил!
Но если б не страдали нравы,
Я балы б до сих пор любил.
Люблю я бешеную младость,
И тесноту, и блеск, и радость,
И дам обдуманный наряд;
Люблю их ножки; только вряд
Найдете вы в России целой
Три пары стройных женских ног.
Ах! долго я забыть не мог
Две ножки… Грустный, охладелый,
Я всё их помню, и во сне
Они тревожат сердце мне.


XXXI


Когда ж и где, в какой пустыне,
Безумец, их забудешь ты?
Ах, ножки, ножки! где вы ныне?
Где мнете вешние цветы?
Взлелеяны в восточной неге,
На северном, печальном снеге
Вы не оставили следов:
Любили мягких вы ковров
Роскошное прикосновенье.
Давно ль для вас я забывал
И жажду славы и похвал,
И край отцов, и заточенье?
Исчезло счастье юных лет,
Как на лугах ваш легкий след.


XXXII


Дианы грудь, ланиты12Флоры
Прелестны, милые друзья!
Однако ножка Терпсихоры
Прелестней чем-то для меня.
Она, пророчествуя взгляду
Неоцененную награду,
Влечет условною красой
Желаний своевольный рой.
Люблю ее, мой друг Эльвина,
Под длинной скатертью столов,
Весной на мураве лугов,
Зимой на чугуне камина,
На зеркальном паркете зал,
У моря на граните скал.


XXXIII


Я помню море пред грозою:
Как я завидовал волнам,
Бегущим бурной чередою
С любовью лечь к ее ногам!
Как я желал тогда с волнами
Коснуться милых ног устами!
Нет, никогда средь пылких дней
Кипящей младости моей
Я не желал с таким мученьем
Лобзать уста младых Армид,
Иль розы пламенных ланит,
Иль перси, полные томленьем;
Нет, никогда порыв страстей
Так не терзал души моей!


XXXIV


Мне памятно другое время!
В заветных иногда мечтах
Держу я счастливое стремя…
И ножку чувствую в руках;
Опять кипит воображенье,
Опять ее прикосновенье
Зажгло в увядшем сердце кровь,
Опять тоска, опять любовь!..
Но полно прославлять надменных
Болтливой лирою своей;
Они не стоят ни страстей,
Ни песен, ими вдохновенных:
Слова и взор волшебниц сих
Обманчивы… как ножки их.


XXXV


Что ж мой Онегин? Полусонный
В постелю с бала едет он:
А Петербург неугомонный
Уж барабаном пробужден.
Встает купец, идет разносчик,
На биржу тянется извозчик,
С кувшином охтенка спешит,
Под ней снег утренний хрустит.
Проснулся утра шум приятный.
Открыты ставни; трубный дым
Столбом восходит голубым,
И хлебник, немец аккуратный,
В бумажном колпаке, не раз
Уж отворял свой васисдас 13. 


XXXVI


Но, шумом бала утомленный,
И утро в полночь обратя,
Спокойно спит в тени блаженной
Забав и роскоши дитя.
Проснется за€ полдень, и снова
До утра жизнь его готова,
Однообразна и пестра,
И завтра то же, что вчера.
Но был ли счастлив мой Евгений,
Свободный, в цвете лучших лет,
Среди блистательных побед,
Среди вседневных наслаждений?
Вотще ли был он средь пиров
Неосторожен и здоров?


XXXVII


Нет: рано чувства в нем остыли;
Ему наскучил света шум;
Красавицы не долго были
Предмет его привычных дум;
Измены утомить успели;
Друзья и дружба надоели,
Затем, что не всегда же мог
Beef-steaks и страсбургский пирог
Шампанской обливать бутылкой
И сыпать острые слова,
Когда болела голова;
И хоть он был повеса пылкой,
Но разлюбил он наконец
И брань, и саблю, и свинец.


XXXVIII


Недуг, которого причину
Давно бы отыскать пора,
Подобный английскому сплину, 
Короче: русская хандра 
Им овладела понемногу;
Он застрелиться, слава Богу,
Попробовать не захотел,
Но к жизни вовсе охладел.
Как Child-Harold, угрюмый, томный
В гостиных появлялся он;
Ни сплетни света, ни бостон,
Ни милый взгляд, ни вздох нескромный,
Ничто не трогало его,
Не замечал он ничего.


XXXIX. XL. XLI


……………………………………
……………………………………
……………………………………


XLII


Причудницы большого света!
Всех прежде вас оставил он;
И правда то, что в наши лета
Довольно скучен высший тон;
Хоть, может быть, иная дама
Толкует Сея и Бентама,
Но вообще их разговор
Несносный, хоть невинный вздор;
К тому ж они так непорочны,
Так величавы, так умны,
Так благочестия полны,
Так осмотрительны, так точны,
Так неприступны для мужчин,
Что вид их уж рождает сплин 14. 


XLIII


И вы, красотки молодые,
Которых позднею порой
Уносят дрожки удалые
По петербургской мостовой,
И вас покинул мой Евгений.
Отступник бурных наслаждений,
Онегин дома заперся,
Зевая, за перо взялся,
Хотел писать – но труд упорный
Ему был тошен; ничего
Не вышло из пера его,
И не попал он в цех задорный
Людей, о коих не сужу,
Затем, что к ним принадлежу.


XLIV


И снова, преданный безделью,
Томясь душевной пустотой,
Уселся он – с похвальной целью
Себе присвоить ум чужой;
Отрядом книг уставил полку,
Читал, читал, а всё без толку:
Там скука, там обман иль бред;
В том совести, в том смысла нет;
На всех различные вериги;
И устарела старина,
И старым бредит новизна.
Как женщин, он оставил книги,
И полку, с пыльной их семьей,
Задернул траурной тафтой.


XLV


Условий света свергнув бремя,
Как он, отстав от суеты,
С ним подружился я в то время.
Мне нравились его черты,
Мечтам невольная преданность,
Неподражательная странность
И резкий, охлажденный ум.
Я был озлоблен, он угрюм;
Страстей игру мы знали оба;
Томила жизнь обоих нас;
В обоих сердца жар угас;
Обоих ожидала злоба
Слепой Фортуны и людей
На самом утре наших дней.


XLVI


Кто жил и мыслил, тот не может
В душе не презирать людей;
Кто чувствовал, того тревожит
Призрак невозвратимых дней:
Тому уж нет очарований,
Того змия воспоминаний,
Того раскаянье грызет.
Всё это часто придает
Большую прелесть разговору.
Сперва Онегина язык
Меня смущал; но я привык
К его язвительному спору,
И к шутке, с желчью пополам,
И злости мрачных эпиграмм.


XLVII


Как часто летнею порою,
Когда прозрачно и светло
Ночное небо над Невою15
И вод веселое стекло
Не отражает лик Дианы,
Воспомня прежних лет романы,
Воспомня прежнюю любовь,
Чувствительны, беспечны вновь,
Дыханьем ночи благосклонной
Безмолвно упивались мы!
Как в лес зеленый из тюрьмы
Перенесен колодник сонный,
Так уносились мы мечтой
К началу жизни молодой.


XLVIII


С душою, полной сожалений,
И опершися на гранит,
Стоял задумчиво Евгений,
Как описал себя пиит16.
Всё было тихо; лишь ночные
Перекликались часовые;
Да дрожек отдаленный стук
С Мильонной17раздавался вдруг;
Лишь лодка, веслами махая,
Плыла по дремлющей реке:
И нас пленяли вдалеке
Рожок и песня удалая…
Но слаще, средь ночных забав,
Напев Торкватовых октав!18


XLIX


Адриатические волны,
О Брента! нет, увижу вас
И, вдохновенья снова полный,
Услышу ваш волшебный глас!
Он свят для внуков Аполлона;
По гордой лире Альбиона19
Он мне знаком, он мне родной.
Ночей Италии златой
Я негой наслажусь на воле
С венецианкою младой,
То говорливой, то немой,
Плывя в таинственной гондоле;
С ней обретут уста мои
Язык Петрарки и любви.


L


Придет ли час моей свободы?
Пора, пора! – взываю к ней;
Брожу над морем20, жду погоды,
Маню ветрила кораблей.
Под ризой бурь, с волнами споря,
По вольному распутью моря
Когда ж начну я вольный бег?
Пора покинуть скучный брег
Мне неприязненной стихии,
И средь полуденных зыбей,
Под небом Африки моей21,
Вздыхать о сумрачной России,
Где я страдал, где я любил,
Где сердце я похоронил.


LI


Онегин был готов со мною
Увидеть чуждые страны;
Но скоро были мы судьбою
На долгий срок разведены.
Отец его тогда скончался.
Перед Онегиным собрался
Заимодавцев жадный полк.
У каждого свой ум и толк:
Евгений, тяжбы ненавидя,
Довольный жребием своим,
Наследство предоставил им,
Большой потери в том не видя
Иль предузнав издалека
Кончину дяди старика.


LII


Вдруг получил он в самом деле
От управителя доклад,
Что дядя при смерти в постеле
И с ним проститься был бы рад.
Прочтя печальное посланье,
Евгений тотчас на свиданье
Стремглав по почте поскакал
И уж заранее зевал,
Приготовляясь, денег ради,
На вздохи, скуку и обман
(И тем я начал мой роман);
Но, прилетев в деревню дяди,
Его нашел уж на столе,
Как дань, готовую земле.


LIII


Нашел он полон двор услуги;
К покойнику со всех сторон
Съезжались недруги и други,
Охотники до похорон.
Покойника похоронили.
Попы и гости ели, пили
И после важно разошлись,
Как будто делом занялись.
Вот наш Онегин – сельский житель,
Заводов, вод, лесов, земель
Хозяин полный, а досель
Порядка враг и расточитель,
И очень рад, что прежний путь
Переменил на что-нибудь.


LIV


Два дня ему казались новы
Уединенные поля,
Прохлада сумрачной дубровы,
Журчанье тихого ручья;
На третий роща, холм и поле
Его не занимали боле;
Потом уж наводили сон;
Потом увидел ясно он,
Что и в деревне скука та же,
Хоть нет ни улиц, ни дворцов,
Ни карт, ни балов, ни стихов.
Хандра ждала его на страже,
И бегала за ним она,
Как тень иль верная жена.


LV


Я был рожден для жизни мирной,
Для деревенской тишины:
В глуши звучнее голос лирный,
Живее творческие сны.
Досугам посвятясь невинным,
Брожу над озером пустынным,
И far niente 22мой закон.
Я каждым утром пробужден
Для сладкой неги и свободы:
Читаю мало, долго сплю,
Летучей славы не ловлю.
Не так ли я в былые годы
Провел в бездействии, в тени
Мои счастливейшие дни?


LVI


Цветы, любовь, деревня, праздность,
Поля! я предан вам душой.
Всегда я рад заметить разность
Между Онегиным и мной,
Чтобы насмешливый читатель
Или какой-нибудь издатель
Замысловатой клеветы,
Сличая здесь мои черты,
Не повторял потом безбожно,
Что намарал я свой портрет,
Как Байрон, гордости поэт,
Как будто нам уж невозможно
Писать поэмы о другом,
Как только о себе самом.


LVII


Замечу кстати: все поэты —
Любви мечтательной друзья.
Бывало, милые предметы
Мне снились, и душа моя
Их образ тайный сохранила;
Их после муза оживила:
Так я, беспечен, воспевал
И деву гор, мой идеал,
И пленниц берегов Салгира23.
Теперь от вас, мои друзья,
Вопрос нередко слышу я:
«О ком твоя вздыхает лира?
Кому, в толпе ревнивых дев,
Ты посвятил ее напев?


LVIII


Чей взор, волнуя вдохновенье,
Умильной лаской наградил
Твое задумчивое пенье?
Кого твой стих боготворил?»
И, други, никого, ей-богу!
Любви безумную тревогу
Я безотрадно испытал.
Блажен, кто с нею сочетал
Горячку рифм: он тем удвоил
Поэзии священный бред,
Петрарке шествуя вослед,
А муки сердца успокоил,
Поймал и славу между тем;
Но я, любя, был глуп и нем.


LIX


Прошла любовь, явилась муза,
И прояснился темный ум.
Свободен, вновь ищу союза
Волшебных звуков, чувств и дум;
Пишу, и сердце не тоскует,
Перо, забывшись, не рисует
Близ неоконченных стихов
Ни женских ножек, ни голов;
Погасший пепел уж не вспыхнет,
Я всё грущу; но слез уж нет,
И скоро, скоро бури след
В душе моей совсем утихнет:
Тогда-то я начну писать
Поэму песен в двадцать пять.


LX


Я думал уж о форме плана
И как героя назову;
Покамест моего романа
Я кончил первую главу;
Пересмотрел всё это строго;
Противоречий очень много,
Но их исправить не хочу;
Цензуре долг свой заплачу
И журналистам на съеденье
Плоды трудов моих отдам;
Иди же к невским берегам,
Новорожденное творенье,
И заслужи мне славы дань:
Кривые толки, шум и брань!



Глава вторая

O rus!
Hor.24
О Русь!



I


Деревня, где скучал Евгений,
Была прелестный уголок;
Там друг невинных наслаждений
Благословить бы небо мог.
Господский дом уединенный,
Горой от ветров огражденный,
Стоял над речкою. Вдали
Пред ним пестрели и цвели
Луга и нивы золотые,
Мелькали сёлы; здесь и там
Стада бродили по лугам,
И сени расширял густые
Огромный, запущенный сад,
Приют задумчивых дриад25.


II


Почтенный замок был построен,
Как замки строиться должны:
Отменно прочен и спокоен
Во вкусе умной старины.
Везде высокие покои,
В гостиной штофные обои,
Царей портреты на стенах,
И печи в пестрых изразцах.
Всё это ныне обветшало,
Не знаю, право, почему;
Да, впрочем, другу моему
В том нужды было очень мало,
Затем, что он равно зевал
Средь модных и старинных зал.


III


Он в том покое поселился,
Где деревенский старожил
Лет сорок с ключницей бранился,
В окно смотрел и мух давил.
Всё было просто: пол дубовый,
Два шкафа, стол, диван пуховый,
Нигде ни пятнышка чернил.
Онегин шкафы отворил;
В одном нашел тетрадь расхода,
В другом наливок целый строй,
Кувшины с яблочной водой
И календарь осьмого года:
Старик, имея много дел,
В иные книги не глядел.


IV


Один среди своих владений,
Чтоб только время проводить,
Сперва задумал наш Евгений
Порядок новый учредить.
В своей глуши мудрец пустынный,
Ярем он барщины старинной
Оброком легким заменил;
И раб судьбу благословил.
Зато в углу своем надулся,
Увидя в этом страшный вред,
Его расчетливый сосед;
Другой лукаво улыбнулся,
И в голос все решили так,
Что он опаснейший чудак.


V


Сначала все к нему езжали;
Но так как с заднего крыльца
Обыкновенно подавали
Ему донского жеребца,
Лишь только вдоль большой дороги
Заслышат их домашни дроги, —
Поступком оскорбясь таким,
Все дружбу прекратили с ним.
«Сосед наш неуч; сумасбродит;
Он фармазон; он пьет одно
Стаканом красное вино;
Он дамам к ручке не подходит;
Всё да да нет; не скажет да-с 
Иль нет-с». Таков был общий глас.


VI


В свою деревню в ту же пору
Помещик новый прискакал
И столь же строгому разбору
В соседстве повод подавал.
По имени Владимир Ленской,
С душою прямо геттингенской26,
Красавец, в полном цвете лет,
Поклонник Канта и поэт.
Он из Германии туманной
Привез учености плоды:
Вольнолюбивые мечты,
Дух пылкий и довольно странный,
Всегда восторженную речь
И кудри черные до плеч.


VII


От хладного разврата света
Еще увянуть не успев,
Его душа была согрета
Приветом друга, лаской дев;
Он сердцем милый был невежда,
Его лелеяла надежда,
И мира новый блеск и шум
Еще пленяли юный ум.
Он забавлял мечтою сладкой
Сомненья сердца своего;
Цель жизни нашей для него
Была заманчивой загадкой,
Над ней он голову ломал
И чудеса подозревал.


VIII


Он верил, что душа родная
Соединиться с ним должна,
Что, безотрадно изнывая,
Его вседневно ждет она;
Он верил, что друзья готовы
За честь его приять оковы
И что не дрогнет их рука
Разбить сосуд клеветника;
Что есть избранные судьбами,
Людей священные друзья;
Что их бессмертная семья
Неотразимыми лучами
Когда-нибудь нас озарит
И мир блаженством одарит.


IX


Негодованье, сожаленье,
Ко благу чистая любовь
И славы сладкое мученье
В нем рано волновали кровь.
Он с лирой странствовал на свете;
Под небом Шиллера и Гете
Их поэтическим огнем
Душа воспламенилась в нем;
И муз возвышенных искусства,
Счастливец, он не постыдил:
Он в песнях гордо сохранил
Всегда возвышенные чувства,
Порывы девственной мечты
И прелесть важной простоты.


X


Он пел любовь, любви послушный,
И песнь его была ясна,
Как мысли девы простодушной,
Как сон младенца, как луна
В пустынях неба безмятежных,
Богиня тайн и вздохов нежных;
Он пел разлуку и печаль,
И нечто, и туманну даль, 
И романтические розы;
Он пел те дальные страны,
Где долго в лоно тишины
Лились его живые слезы;
Он пел поблеклый жизни цвет
Без малого в осьмнадцать лет.


XI


В пустыне, где один Евгений
Мог оценить его дары,
Господ соседственных селений
Ему не нравились пиры;
Бежал он их беседы шумной,
Их разговор благоразумный
О сенокосе, о вине,
О псарне, о своей родне,
Конечно, не блистал ни чувством,
Ни поэтическим огнем,
Ни остротою, ни умом,
Ни общежития искусством;
Но разговор их милых жен
Гораздо меньше был умен.


XII


Богат, хорош собою, Ленский
Везде был принят как жених;
Таков обычай деревенский;
Все дочек прочили своих
За полурусского соседа; 
Взойдет ли он, тотчас беседа
Заводит слово стороной
О скуке жизни холостой;
Зовут соседа к самовару,
А Дуня разливает чай,
Ей шепчут: «Дуня, примечай!»
Потом приносят и гитару;
И запищит она (Бог мой!):
Приди в чертог ко мне златой !..27


XIII


Но Ленский, не имев, конечно,
Охоты узы брака несть,
С Онегиным желал сердечно
Знакомство покороче свесть.
Они сошлись. Волна и камень,
Стихи и проза, лед и пламень
Не столь различны меж собой.
Сперва взаимной разнотой
Они друг другу были скучны;
Потом понравились; потом
Съезжались каждый день верхом
И скоро стали неразлучны.
Так люди (первый каюсь я)
От делать нечего друзья.


XIV


Но дружбы нет и той меж нами.
Все предрассудки истребя,
Мы почитаем всех нулями,
А единицами – себя.
Мы все глядим в Наполеоны;
Двуногих тварей миллионы
Для нас орудие одно,
Нам чувство дико и смешно.
Сноснее многих был Евгений;
Хоть он людей, конечно, знал
И вообще их презирал, —
Но (правил нет без исключений)
Иных он очень отличал
И вчуже чувство уважал.


XV


Он слушал Ленского с улыбкой.
Поэта пылкий разговор,
И ум, еще в сужденьях зыбкой,
И вечно вдохновенный взор, —
Онегину всё было ново;
Он охладительное слово
В устах старался удержать
И думал: глупо мне мешать
Его минутному блаженству;
И без меня пора придет,
Пускай покамест он живет
Да верит мира совершенству;
Простим горячке юных лет
И юный жар и юный бред.


XVI


Меж ими всё рождало споры
И к размышлению влекло:
Племен минувших договоры,
Плоды наук, добро и зло,
И предрассудки вековые,
И гроба тайны роковые,
Судьба и жизнь в свою чреду, —
Всё подвергалось их суду.
Поэт в жару своих суждений
Читал, забывшись, между тем
Отрывки северных поэм,
И снисходительный Евгений,
Хоть их не много понимал,
Прилежно юноше внимал.


XVII


Но чаще занимали страсти
Умы пустынников моих.
Ушед от их мятежной власти,
Онегин говорил об них
С невольным вздохом сожаленья;
Блажен, кто ведал их волненья
И наконец от них отстал;
Блаженней тот, кто их не знал,
Кто охлаждал любовь – разлукой,
Вражду – злословием; порой
Зевал с друзьями и с женой,
Ревнивой не тревожась мукой,
И дедов верный капитал
Коварной двойке не вверял.


XVIII


Когда прибегнем мы под знамя
Благоразумной тишины,
Когда страстей угаснет пламя
И нам становятся смешны
Их своевольство иль порывы
И запоздалые отзывы, —
Смиренные не без труда,
Мы любим слушать иногда
Страстей чужих язык мятежный,
И нам он сердце шевелит.
Так точно старый инвалид
Охотно клонит слух прилежный
Рассказам юных усачей,
Забытый в хижине своей.


XIX


Зато и пламенная младость
Не может ничего скрывать.
Вражду, любовь, печаль и радость
Она готова разболтать.
В любви считаясь инвалидом,
Онегин слушал с важным видом,
Как, сердца исповедь любя,
Поэт высказывал себя;
Свою доверчивую совесть
Он простодушно обнажал.
Евгений без труда узнал
Его любви младую повесть,
Обильный чувствами рассказ,
Давно не новыми для нас.


XX


Ах, он любил, как в наши лета
Уже не любят; как одна
Безумная душа поэта
Еще любить осуждена:
Всегда, везде одно мечтанье,
Одно привычное желанье,
Одна привычная печаль.
Ни охлаждающая даль,
Ни долгие лета разлуки,
Ни музам данные часы,
Ни чужеземные красы,
Ни шум веселий, ни науки
Души не изменили в нем,
Согретой девственным огнем.


XXI


Чуть отрок, Ольгою плененный,
Сердечных мук еще не знав,
Он был свидетель умиленный
Ее младенческих забав;
В тени хранительной дубравы
Он разделял ее забавы,
И детям прочили венцы
Друзья-соседи, их отцы.
В глуши, под сению смиренной,
Невинной прелести полна,
В глазах родителей, она
Цвела как ландыш потаенный,
Не знаемый в траве глухой
Ни мотыльками, ни пчелой.


XXII


Она поэту подарила
Младых восторгов первый сон,
И мысль об ней одушевила
Его цевницы первый стон.
Простите, игры золотые!
Он рощи полюбил густые,
Уединенье, тишину,
И ночь, и звезды, и луну,
Луну, небесную лампаду,
Которой посвящали мы
Прогулки средь вечерней тьмы,
И слезы, тайных мук отраду…
Но нынче видим только в ней
Замену тусклых фонарей.


XXIII


Всегда скромна, всегда послушна,
Всегда как утро весела,
Как жизнь поэта простодушна,
Как поцелуй любви мила,
Глаза как небо голубые;
Улыбка, локоны льняные,
Движенья, голос, легкий стан —
Всё в Ольге… но любой роман
Возьмите и найдете, верно,
Ее портрет: он очень мил,
Я прежде сам его любил,
Но надоел он мне безмерно.
Позвольте мне, читатель мой,
Заняться старшею сестрой.


XXIV


Ее сестра звалась Татьяна…28
Впервые именем таким
Страницы нежные романа
Мы своевольно освятим.
И что ж? оно приятно, звучно;
Но с ним, я знаю, неразлучно
Воспоминанье старины
Иль девичьей! Мы все должны
Признаться: вкусу очень мало
У нас и в наших именах
(Не говорим уж о стихах);
Нам просвещенье не пристало,
И нам досталось от него
Жеманство, – больше ничего.


XXV


Итак, она звалась Татьяной.
Ни красотой сестры своей,
Ни свежестью ее румяной
Не привлекла б она очей.
Дика, печальна, молчалива,
Как лань лесная, боязлива,
Она в семье своей родной
Казалась девочкой чужой.
Она ласкаться не умела
К отцу, ни к матери своей;
Дитя сама, в толпе детей
Играть и прыгать не хотела
И часто целый день одна
Сидела молча у окна.


XXVI


Задумчивость, ее подруга
От самых колыбельных дней,
Теченье сельского досуга
Мечтами украшала ей.
Ее изнеженные пальцы
Не знали игл; склонясь на пяльцы,
Узором шелковым она
Не оживляла полотна.
Охоты властвовать примета,
С послушной куклою дитя
Приготовляется шутя
К приличию, закону света,
И важно повторяет ей
Уроки маменьки своей.


XXVII


Но куклы даже в эти годы
Татьяна в руки не брала;
Про вести города, про моды
Беседы с нею не вела.
И были детские проказы
Ей чужды: страшные рассказы
Зимою в темноте ночей
Пленяли больше сердце ей.
Когда же няня собирала
Для Ольги на широкий луг
Всех маленьких ее подруг,
Она в горелки не играла,
Ей скучен был и звонкий смех,
И шум их ветреных утех.


XXVIII


Она любила на балконе
Предупреждать зари восход,
Когда на бледном небосклоне
Звезд исчезает хоровод,
И тихо край земли светлеет,
И, вестник утра, ветер веет,
И всходит постепенно день.
Зимой, когда ночная тень
Полмиром доле обладает,
И доле в праздной тишине,
При отуманенной луне,
Восток ленивый почивает,
В привычный час пробуждена
Вставала при свечах она.


XXIX


Ей рано нравились романы;
Они ей заменяли всё;
Она влюблялася в обманы
И Ричардсона и Руссо.
Отец ее был добрый малый,
В прошедшем веке запоздалый;
Но в книгах не видал вреда;
Он, не читая никогда,
Их почитал пустой игрушкой
И не заботился о том,
Какой у дочки тайный том
Дремал до утра под подушкой.
Жена ж его была сама
От Ричардсона без ума.


XXX


Она любила Ричардсона
Не потому, чтобы прочла,
Не потому, чтоб Грандисона
Она Ловласу предпочла29;
Но в старину княжна Алина,
Ее московская кузина,
Твердила часто ей об них.
В то время был еще жених
Ее супруг, но по неволе;
Она вздыхала о другом,
Который сердцем и умом
Ей нравился гораздо боле:
Сей Грандисон был славный франт,
Игрок и гвардии сержант.


XXXI


Как он, она была одета
Всегда по моде и к лицу;
Но, не спросясь ее совета,
Девицу повезли к венцу.
И, чтоб ее рассеять горе,
Разумный муж уехал вскоре
В свою деревню, где она,
Бог знает кем окружена,
Рвалась и плакала сначала,
С супругом чуть не развелась;
Потом хозяйством занялась,
Привыкла и довольна стала.
Привычка свыше нам дана:
Замена счастию она30.


XXXII


Привычка усладила горе,
Не отразимое ничем;
Открытие большое вскоре
Ее утешило совсем:
Она меж делом и досугом
Открыла тайну, как супругом
Самодержавно управлять,
И всё тогда пошло на стать.
Она езжала по работам,
Солила на зиму грибы,
Вела расходы, брила лбы,
Ходила в баню по субботам,
Служанок била осердясь —
Всё это мужа не спросясь.


XXXIII


Бывало, писывала кровью
Она в альбомы нежных дев,
Звала Полиною Прасковью
И говорила нараспев,
Корсет носила очень узкий,
И русский Н, как N французский,
Произносить умела в нос;
Но скоро всё перевелось;
Корсет, альбом, княжну Алину,
Стишков чувствительных тетрадь
Она забыла; стала звать
Акулькой прежнюю Селину
И обновила наконец
На вате шлафор и чепец.


XXXIV


Но муж любил ее сердечно,
В ее затеи не входил,
Во всем ей веровал беспечно,
А сам в халате ел и пил;
Покойно жизнь его катилась;
Под вечер иногда сходилась
Соседей добрая семья,
Нецеремонные друзья,
И потужить, и позлословить,
И посмеяться кой о чем.
Проходит время; между тем
Прикажут Ольге чай готовить,
Там ужин, там и спать пора,
И гости едут со двора.


XXXV


Они хранили в жизни мирной
Привычки милой старины;
У них на масленице жирной
Водились русские блины;
Два раза в год они говели;
Любили круглые качели,
Подблюдны песни, хоровод;
В день Троицын, когда народ
Зевая слушает молебен,
Умильно на пучок зари
Они роняли слезки три;
Им квас как воздух был потребен,
И за столом у них гостям
Носили блюда по чинам.


XXXVI


И так они старели оба.
И отворились наконец
Перед супругом двери гроба,
И новый он приял венец.
Он умер в час перед обедом,
Оплаканный своим соседом,
Детьми и верною женой
Чистосердечней, чем иной.
Он был простой и добрый барин,
И там, где прах его лежит,
Надгробный памятник гласит:
Смиренный грешник, Дмитрий Ларин, 
Господний раб и бригадир, 
Под камнем сим вкушает мир. 


XXXVII


Своим пенатам возвращенный,
Владимир Ленский посетил
Соседа памятник смиренный,
И вздох он пеплу посвятил;
И долго сердцу грустно было.
«Poor Yorick! 31 – молвил он уныло, —
Он на руках меня держал.
Как часто в детстве я играл
Его Очаковской медалью!
Он Ольгу прочил за меня,
Он говорил: дождусь ли дня?..»
И, полный искренней печалью,
Владимир тут же начертал
Ему надгробный мадригал.


XXXVIII


И там же надписью печальной
Отца и матери, в слезах,
Почтил он прах патриархальный…
Увы! на жизненных браздах
Мгновенной жатвой поколенья,
По тайной воле провиденья,
Восходят, зреют и падут;
Другие им вослед идут…
Так наше ветреное племя
Растет, волнуется, кипит
И к гробу прадедов теснит.
Придет, придет и наше время,
И наши внуки в добрый час
Из мира вытеснят и нас!


XXXIX


Покамест упивайтесь ею,
Сей легкой жизнию, друзья!
Ее ничтожность разумею
И мало к ней привязан я;
Для призраков закрыл я вежды;
Но отдаленные надежды
Тревожат сердце иногда:
Без неприметного следа
Мне было б грустно мир оставить.
Живу, пишу не для похвал;
Но я бы, кажется, желал
Печальный жребий свой прославить,
Чтоб обо мне, как верный друг,
Напомнил хоть единый звук.


XL


И чье-нибудь он сердце тронет;
И, сохраненная судьбой,
Быть может, в Лете не потонет
Строфа, слагаемая мной;
Быть может (лестная надежда!),
Укажет будущий невежда
На мой прославленный портрет
И молвит: то-то был поэт!
Прими ж мои благодаренья,
Поклонник мирных аонид,
О ты, чья память сохранит
Мои летучие творенья,
Чья благосклонная рука
Потреплет лавры старика!



Глава третья

Elle e€tait fille, elle e€tait amoureuse.
Malfila^tre 32



I


«Куда? Уж эти мне поэты!»
– Прощай, Онегин, мне пора.
«Я не держу тебя; но где ты
Свои проводишь вечера?»
– У Лариных. – «Вот это чудно.
Помилуй! и тебе не трудно
Там каждый вечер убивать?»
– Нимало. – «Не могу понять.
Отселе вижу, что такое:
Во-первых (слушай, прав ли я?),
Простая, русская семья,
К гостям усердие большое,
Варенье, вечный разговор
Про дождь, про лён, про скотный двор…»


II


– Я тут еще беды не вижу.
«Да скука, вот беда, мой друг».
– Я модный свет ваш ненавижу;
Милее мне домашний круг,
Где я могу… – «Опять эклога!33
Да полно, милый, ради Бога.
Ну что ж? ты едешь: очень жаль.
Ах, слушай, Ленский; да нельзя ль
Увидеть мне Филлиду эту,
Предмет и мыслей, и пера,
И слез, и рифм et cetera?.. 
Представь меня». – «Ты шутишь». – «Нету».
– Я рад. – «Когда же?» – Хоть сейчас
Они с охотой примут нас.


III


Поедем. —
Поскакали други,
Явились; им расточены
Порой тяжелые услуги
Гостеприимной старины.
Обряд известный угощенья:
Несут на блюдечках варенья,
На столик ставят вощаной
Кувшин с брусничною водой.
……………………………………


IV


Они дорогой самой краткой
Домой летят во весь опор34.
Теперь послушаем украдкой
Героев наших разговор:
– Ну что ж, Онегин? ты зеваешь. —
«Привычка, Ленский». – Но скучаешь
Ты как-то больше. – «Нет, равно.
Однако в поле уж темно;
Скорей! пошел, пошел, Андрюшка!
Какие глупые места!
А кстати: Ларина проста,
Но очень милая старушка;
Боюсь: брусничная вода
Мне не наделала б вреда.


V


Скажи: которая Татьяна?» —
«Да та, которая грустна
И молчалива, как Светлана,
Вошла и села у окна». —
«Неужто ты влюблен в меньшую?» —
«А что?» – «Я выбрал бы другую,
Когда б я был, как ты, поэт.
В чертах у Ольги жизни нет,
Точь-в-точь в Вандиковой Мадонне:
Кругла, красна лицом она,
Как эта глупая луна
На этом глупом небосклоне».
Владимир сухо отвечал
И после во весь путь молчал.


VI


Меж тем Онегина явленье
У Лариных произвело
На всех большое впечатленье
И всех соседей развлекло.
Пошла догадка за догадкой.
Все стали толковать украдкой,
Шутить, судить не без греха,
Татьяне прочить жениха;
Иные даже утверждали,
Что свадьба слажена совсем,
Но остановлена затем,
Что модных колец не достали.
О свадьбе Ленского давно
У них уж было решено.


VII


Татьяна слушала с досадой
Такие сплетни; но тайком
С неизъяснимою отрадой
Невольно думала о том;
И в сердце дума заронилась;
Пора пришла, она влюбилась.
Так в землю падшее зерно
Весны огнем оживлено.
Давно ее воображенье,
Сгорая негой и тоской,
Алкало пищи роковой;
Давно сердечное томленье
Теснило ей младую грудь;
Душа ждала… кого-нибудь,


VIII


И дождалась… Открылись очи;
Она сказала: это он!
Увы! теперь и дни, и ночи,
И жаркий одинокий сон,
Всё полно им; всё деве милой
Без умолку волшебной силой
Твердит о нем. Докучны ей
И звуки ласковых речей,
И взор заботливой прислуги.
В уныние погружена,
Гостей не слушает она
И проклинает их досуги,
Их неожиданный приезд
И продолжительный присест.


IX


Теперь с каким она вниманьем
Читает сладостный роман,
С каким живым очарованьем
Пьет обольстительный обман!
Счастливой силою мечтанья
Одушевленные созданья,
Любовник Юлии Вольмар,
Малек-Адель и де Линар,
И Вертер, мученик мятежный,
И бесподобный Грандисон35,
Который нам наводит сон, —
Все для мечтательницы нежной
В единый образ облеклись,
В одном Онегине слились.


X


Воображаясь героиней
Своих возлюбленных творцов,
Кларисой, Юлией, Дельфиной,
Татьяна в тишине лесов
Одна с опасной книгой бродит,
Она в ней ищет и находит
Свой тайный жар, свои мечты,
Плоды сердечной полноты,
Вздыхает и, себе присвоя
Чужой восторг, чужую грусть,
В забвенье шепчет наизусть
Письмо для милого героя…
Но наш герой, кто б ни был он,
Уж верно был не Грандисон.


XI


Свой слог на важный лад настроя,
Бывало, пламенный творец
Являл нам своего героя
Как совершенства образец.
Он одарял предмет любимый,
Всегда неправедно гонимый,
Душой чувствительной, умом
И привлекательным лицом.
Питая жар чистейшей страсти,
Всегда восторженный герой
Готов был жертвовать собой,
И при конце последней части
Всегда наказан был порок,
Добру достойный был венок.


XII


А нынче все умы в тумане,
Мораль на нас наводит сон,
Порок любезен и в романе,
И там уж торжествует он.
Британской музы небылицы
Тревожат сон отроковицы,
И стал теперь ее кумир
Или задумчивый Вампир,
Или Мельмот, бродяга мрачный,
Иль Вечный жид, или Корсар,
Или таинственный Сбогар36.
Лорд Байрон прихотью удачной
Облек в унылый романтизм
И безнадежный эгоизм.


XIII


Друзья мои, что ж толку в этом?
Быть может, волею небес,
Я перестану быть поэтом,
В меня вселится новый бес,
И, Фебовы презрев угрозы,
Унижусь до смиренной прозы;
Тогда роман на старый лад
Займет веселый мой закат.
Не муки тайные злодейства
Я грозно в нем изображу,
Но просто вам перескажу
Преданья русского семейства,
Любви пленительные сны
Да нравы нашей старины.


XIV


Перескажу простые речи
Отца иль дяди-старика,
Детей условленные встречи
У старых лип, у ручейка;
Несчастной ревности мученья,
Разлуку, слезы примиренья,
Поссорю вновь, и наконец
Я поведу их под венец…
Я вспомню речи неги страстной,
Слова тоскующей любви,
Которые в минувши дни
У ног любовницы прекрасной
Мне приходили на язык,
От коих я теперь отвык.


XV


Татьяна, милая Татьяна!
С тобой теперь я слезы лью;
Ты в руки модного тирана
Уж отдала судьбу свою.
Погибнешь, милая; но прежде
Ты в ослепительной надежде
Блаженство темное зовешь,
Ты негу жизни узнаешь,
Ты пьешь волшебный яд желаний,
Тебя преследуют мечты:
Везде воображаешь ты
Приюты счастливых свиданий;
Везде, везде перед тобой
Твой искуситель роковой.


XVI


Тоска любви Татьяну гонит,
И в сад идет она грустить,
И вдруг недвижны очи клонит,
И лень ей далее ступить.
Приподнялася грудь, ланиты
Мгновенным пламенем покрыты,
Дыханье замерло в устах,
И в слухе шум, и блеск в очах…
Настанет ночь; луна обходит
Дозором дальный свод небес,
И соловей во мгле древес
Напевы звучные заводит.
Татьяна в темноте не спит
И тихо с няней говорит:


XVII


«Не спится, няня: здесь так душно!
Открой окно да сядь ко мне». —
«Что, Таня, что с тобой?» – «Мне скучно,
Поговорим о старине». —
«О чем же, Таня? Я, бывало,
Хранила в памяти не мало
Старинных былей, небылиц
Про злых духов и про девиц;
А нынче всё мне тёмно, Таня:
Что знала, то забыла. Да,
Пришла худая череда!
Зашибло…» – «Расскажи мне, няня,
Про ваши старые года:
Была ты влюблена тогда?» —


XVIII


«И полно, Таня! В эти лета
Мы не слыхали про любовь;
А то бы согнала со света
Меня покойница свекровь». —
«Да как же ты венчалась, няня?» —
«Так, видно, Бог велел. Мой Ваня
Моложе был меня, мой свет,
А было мне тринадцать лет.
Недели две ходила сваха
К моей родне, и наконец
Благословил меня отец.
Я горько плакала со страха,
Мне с плачем косу расплели
Да с пеньем в церковь повели.


XIX


И вот ввели в семью чужую…
Да ты не слушаешь меня…» —
«Ах, няня, няня, я тоскую,
Мне тошно, милая моя:
Я плакать, я рыдать готова!..» —
«Дитя мое, ты нездорова;
Господь помилуй и спаси!
Чего ты хочешь, попроси…
Дай окроплю святой водою,
Ты вся горишь…» – «Я не больна:
Я… знаешь, няня… влюблена».
«Дитя мое, Господь с тобою!» —
И няня девушку с мольбой
Крестила дряхлою рукой.


XX


«Я влюблена», – шептала снова
Старушке с горестью она.
«Сердечный друг, ты нездорова». —
«Оставь меня: я влюблена».
И между тем луна сияла
И томным светом озаряла
Татьяны бледные красы,
И распущенные власы,
И капли слез, и на скамейке
Пред героиней молодой,
С платком на голове седой,
Старушку в длинной телогрейке:
И всё дремало в тишине
При вдохновительной луне.


XXI


И сердцем далеко носилась
Татьяна, смотря на луну…
Вдруг мысль в уме ее родилась…
«Поди, оставь меня одну.
Дай, няня, мне перо, бумагу
Да стол подвинь; я скоро лягу;
Прости». И вот она одна.
Всё тихо. Светит ей луна.
Облокотясь, Татьяна пишет.
И всё Евгений на уме,
И в необдуманном письме
Любовь невинной девы дышит.
Письмо готово, сложено…
Татьяна! для кого ж оно?


XXII


Я знал красавиц недоступных,
Холодных, чистых, как зима,
Неумолимых, неподкупных,
Непостижимых для ума;
Дивился я их спеси модной,
Их добродетели природной,
И, признаюсь, от них бежал,
И, мнится, с ужасом читал
Над их бровями надпись ада:
Оставь надежду навсегда 37. 
Внушать любовь для них беда,
Пугать людей для них отрада.
Быть может, на брегах Невы
Подобных дам видали вы.


XXIII


Среди поклонников послушных
Других причудниц я видал,
Самолюбиво равнодушных
Для вздохов страстных и похвал.
И что ж нашел я с изумленьем?
Они, суровым поведеньем
Пугая робкую любовь,
Ее привлечь умели вновь,
По крайней мере сожаленьем,
По крайней мере звук речей
Казался иногда нежней,
И с легковерным ослепленьем
Опять любовник молодой
Бежал за милой суетой.


XXIV


За что ж виновнее Татьяна?
За то ль, что в милой простоте
Она не ведает обмана
И верит избранной мечте?
За то ль, что любит без искусства,
Послушная влеченью чувства,
Что так доверчива она,
Что от небес одарена
Воображением мятежным,
Умом и волею живой,
И своенравной головой,
И сердцем пламенным и нежным?
Ужели не простите ей
Вы легкомыслия страстей?


XXV


Кокетка судит хладнокровно,
Татьяна любит не шутя
И предается безусловно
Любви, как милое дитя.
Не говорит она: отложим —
Любви мы цену тем умножим,
Вернее в сети заведем;
Сперва тщеславие кольнем
Надеждой, там недоуменьем
Измучим сердце, а потом
Ревнивым оживим огнем;
А то, скучая наслажденьем,
Невольник хитрый из оков
Всечасно вырваться готов.


XXVI


Еще предвижу затрудненья:
Родной земли спасая честь,
Я должен буду, без сомненья,
Письмо Татьяны перевесть.
Она по-русски плохо знала,
Журналов наших не читала,
И выражалася с трудом
На языке своем родном,
Итак, писала по-французски…
Что делать! повторяю вновь:
Доныне дамская любовь
Не изъяснялася по-русски,
Доныне гордый наш язык
К почтовой прозе не привык.


XXVII


Я знаю: дам хотят заставить
Читать по-русски. Право, страх!
Могу ли их себе представить
С «Благонамеренным»38в руках!
Я шлюсь на вас, мои поэты;
Не правда ль: милые предметы,
Которым, за свои грехи,
Писали втайне вы стихи,
Которым сердце посвящали,
Не все ли, русским языком
Владея слабо и с трудом,
Его так мило искажали,
И в их устах язык чужой
Не обратился ли в родной?


XXVIII


Не дай мне Бог сойтись на бале
Иль при разъезде на крыльце
С семинаристом в желтой шале
Иль с академиком в чепце!
Как уст румяных без улыбки,
Без грамматической ошибки
Я русской речи не люблю.
Быть может, на беду мою,
Красавиц новых поколенье,
Журналов вняв молящий глас,
К грамматике приучит нас;
Стихи введут в употребленье;
Но я… какое дело мне?
Я верен буду старине.


XXIX


Неправильный, небрежный лепет,
Неточный выговор речей
По-прежнему сердечный трепет
Произведут в груди моей;
Раскаяться во мне нет силы,
Мне галлицизмы39будут милы,
Как прошлой юности грехи,
Как Богдановича стихи.
Но полно. Мне пора заняться
Письмом красавицы моей;
Я слово дал, и что ж? ей-ей,
Теперь готов уж отказаться.
Я знаю: нежного Парни
Перо не в моде в наши дни.


XXX


Певец Пиров и грусти томной40,
Когда б еще ты был со мной,
Я стал бы просьбою нескромной
Тебя тревожить, милый мой:
Чтоб на волшебные напевы
Переложил ты страстной девы
Иноплеменные слова.
Где ты? приди: свои права
Передаю тебе с поклоном…
Но посреди печальных скал,
Отвыкнув сердцем от похвал,
Один, под финским небосклоном,
Он бродит, и душа его
Не слышит горя моего.


XXXI


Письмо Татьяны предо мною;
Его я свято берегу,
Читаю с тайною тоскою
И начитаться не могу.
Кто ей внушал и эту нежность,
И слов любезную небрежность?
Кто ей внушал умильный вздор,
Безумный сердца разговор,
И увлекательный и вредный?
Я не могу понять. Но вот
Неполный, слабый перевод,
С живой картины список бледный,
Или разыгранный Фрейшиц
Перстами робких учениц:


Письмо Татьяны к Онегину

Я к вам пишу – чего же боле?
Что я могу еще сказать?
Теперь, я знаю, в вашей воле
Меня презреньем наказать.
Но вы, к моей несчастной доле
Хоть каплю жалости храня,
Вы не оставите меня.
Сначала я молчать хотела;
Поверьте: моего стыда
Вы не узнали б никогда,
Когда б надежду я имела
Хоть редко, хоть в неделю раз
В деревне нашей видеть вас,
Чтоб только слышать ваши речи,
Вам слово молвить, и потом
Всё думать, думать об одном
И день и ночь до новой встречи.
Но говорят, вы нелюдим;
В глуши, в деревне всё вам скучно,
А мы… ничем мы не блестим,
Хоть вам и рады простодушно.

Зачем вы посетили нас?
В глуши забытого селенья
Я никогда не знала б вас,
Не знала б горького мученья.
Души неопытной волненья
Смирив со временем (как знать?),
По сердцу я нашла бы друга,
Была бы верная супруга
И добродетельная мать.
Другой!.. Нет, никому на свете
Не отдала бы сердца я!
То в вышнем суждено совете…
То воля неба: я твоя;
Вся жизнь моя была залогом
Свиданья верного с тобой;
Я знаю, ты мне послан Богом,
До гроба ты хранитель мой…
Ты в сновиденьях мне являлся,
Незримый, ты мне был уж мил,
Твой чудный взгляд меня томил,
В душе твой голос раздавался
Давно… нет, это был не сон!
Ты чуть вошел, я вмиг узнала,
Вся обомлела, запылала
И в мыслях молвила: вот он!
Не правда ль? я тебя слыхала:
Ты говорил со мной в тиши,
Когда я бедным помогала
Или молитвой услаждала
Тоску волнуемой души?
И в это самое мгновенье
Не ты ли, милое виденье,
В прозрачной темноте мелькнул,
Приникнул тихо к изголовью?
Не ты ль, с отрадой и любовью,
Слова надежды мне шепнул?
Кто ты, мой ангел ли хранитель
Или коварный искуситель:
Мои сомненья разреши.
Быть может, это всё пустое,
Обман неопытной души!
И суждено совсем иное…
Но так и быть! Судьбу мою
Отныне я тебе вручаю,
Перед тобою слезы лью,
Твоей защиты умоляю…
Вообрази: я здесь одна,
Никто меня не понимает,
Рассудок мой изнемогает,
И молча гибнуть я должна.
Я жду тебя: единым взором
Надежды сердца оживи
Иль сон тяжелый перерви,
Увы, заслуженным укором!
Кончаю! Страшно перечесть…
Стыдом и страхом замираю…
Но мне порукой ваша честь,
И смело ей себя вверяю…


XXXII


Татьяна то вздохнет, то охнет;
Письмо дрожит в ее руке;
Облатка розовая сохнет
На воспаленном языке.
К плечу головушкой склонилась.
Сорочка легкая спустилась
С ее прелестного плеча…
Но вот уж лунного луча
Сиянье гаснет. Там долина
Сквозь пар яснеет. Там поток
Засеребрился; там рожок
Пастуший будит селянина.
Вот утро: встали все давно,
Моей Татьяне всё равно.


XXXIII


Она зари не замечает,
Сидит с поникшею главой
И на письмо не напирает
Своей печати вырезной.
Но, дверь тихонько отпирая,
Уж ей Филипьевна седая
Приносит на подносе чай.
«Пора, дитя мое, вставай:
Да ты, красавица, готова!
О пташка ранняя моя!
Вечор уж как боялась я!
Да, слава Богу, ты здорова!
Тоски ночной и следу нет,
Лицо твое как маков цвет». —


XXXIV


«Ах! няня, сделай одолженье». —
«Изволь, родная, прикажи».
«Не думай… право… подозренье…
Но видишь… ах! не откажи». —
«Мой друг, вот Бог тебе порука». —
«Итак, пошли тихонько внука
С запиской этой к О… к тому…
К соседу… да велеть ему,
Чтоб он не говорил ни слова,
Чтоб он не называл меня…» —
«Кому же, милая моя?
Я нынче стала бестолкова.
Кругом соседей много есть;
Куда мне их и перечесть». —


XXXV


«Как недогадлива ты, няня!» —
«Сердечный друг, уж я стара,
Стара; тупеет разум, Таня;
А то, бывало, я востра,
Бывало, слово барской воли…» —
«Ах, няня, няня! до того ли?
Что нужды мне в твоем уме?
Ты видишь, дело о письме
К Онегину». – «Ну, дело, дело.
Не гневайся, душа моя,
Ты знаешь, непонятна я…
Да что ж ты снова побледнела?» —
«Так, няня, право, ничего.
Пошли же внука своего». —


XXXVI


Но день протек, и нет ответа.
Другой настал: всё нет, как нет.
Бледна как тень, с утра одета,
Татьяна ждет: когда ж ответ?
Приехал Ольгин обожатель.
«Скажите: где же ваш приятель? —
Ему вопрос хозяйки был. —
Он что-то нас совсем забыл».
Татьяна, вспыхнув, задрожала.
«Сегодня быть он обещал, —
Старушке Ленский отвечал, —
Да, видно, почта задержала». —
Татьяна потупила взор,
Как будто слыша злой укор.


XXXVII


Смеркалось; на столе, блистая,
Шипел вечерний самовар,
Китайский чайник нагревая;
Под ним клубился легкий пар.
Разлитый Ольгиной рукою,
По чашкам темною струею
Уже душистый чай бежал,
И сливки мальчик подавал;
Татьяна пред окном стояла,
На стекла хладные дыша,
Задумавшись, моя душа,
Прелестным пальчиком писала
На отуманенном стекле
Заветный вензель О да Е. 


XXXVIII


И между тем душа в ней ныла,
И слез был полон томный взор.
Вдруг топот!.. кровь ее застыла.
Вот ближе! скачут… и на двор
Евгений! «Ах!» – и легче тени
Татьяна прыг в другие сени,
С крыльца на двор, и прямо в сад,
Летит, летит; взглянуть назад
Не смеет; мигом обежала
Куртины, мостики, лужок,
Аллею к озеру, лесок,
Кусты сирен переломала,
По цветникам летя к ручью,
И, задыхаясь, на скамью


XXXIX


Упала…
«Здесь он! здесь Евгений!
О Боже! что подумал он!»
В ней сердце, полное мучений,
Хранит надежды темный сон;
Она дрожит и жаром пышет,
И ждет: нейдет ли? Но не слышит.
В саду служанки, на грядах,
Сбирали ягоду в кустах
И хором по наказу пели
(Наказ, основанный на том,
Чтоб барской ягоды тайком
Уста лукавые не ели
И пеньем были заняты:
Затея сельской остроты!).

Песня девушек

Девицы, красавицы,
Душеньки, подруженьки,
Разыграйтесь, девицы,
Разгуляйтесь, милые!
Затяните песенку,
Песенку заветную,
Заманите молодца
К хороводу нашему.
Как заманим молодца,
Как завидим издали,
Разбежимтесь, милые,
Закидаем вишеньем,
Вишеньем, малиною,
Красною смородиной.
Не ходи подслушивать
Песенки заветные,
Не ходи подсматривать
Игры наши девичьи.


XL


Они поют, и, с небреженьем
Внимая звонкий голос их,
Ждала Татьяна с нетерпеньем,
Чтоб трепет сердца в ней затих,
Чтобы прошло ланит пыланье.
Но в персях то же трепетанье,
И не проходит жар ланит,
Но ярче, ярче лишь горит…
Так бедный мотылек и блещет,
И бьется радужным крылом,
Плененный школьным шалуном;
Так зайчик в озими трепещет,
Увидя вдруг издалека
В кусты припадшего стрелка.


XLI


Но наконец она вздохнула
И встала со скамьи своей;
Пошла, но только повернула
В аллею, прямо перед ней,
Блистая взорами, Евгений
Стоит подобно грозной тени,
И, как огнем обожжена,
Остановилася она.
Но следствия нежданной встречи
Сегодня, милые друзья,
Пересказать не в силах я;
Мне должно после долгой речи
И погулять и отдохнуть:
Докончу после как-нибудь.



Глава четвертая

La morale est dans la nature des choses.
Necker 41



I. II. III. IV. V. VI



VII


Чем меньше женщину мы любим,
Тем легче нравимся мы ей
И тем ее вернее губим
Средь обольстительных сетей.
Разврат, бывало, хладнокровный
Наукой славился любовной,
Сам о себе везде трубя
И наслаждаясь не любя.
Но эта важная забава
Достойна старых обезьян
Хваленых дедовских времян:
Ловласов обветшала слава
Со славой красных каблуков
И величавых париков.


VIII


Кому не скучно лицемерить,
Различно повторять одно,
Стараться важно в том уверить,
В чем все уверены давно,
Всё те же слышать возраженья,
Уничтожать предрассужденья,
Которых не было и нет
У девочки в тринадцать лет!
Кого не утомят угрозы,
Моленья, клятвы, мнимый страх,
Записки на шести листах,
Обманы, сплетни, кольцы, слезы,
Надзоры теток, матерей,
И дружба тяжкая мужей!


IX


Так точно думал мой Евгений.
Он в первой юности своей
Был жертвой бурных заблуждений
И необузданных страстей.
Привычкой жизни избалован,
Одним на время очарован,
Разочарованный другим,
Желаньем медленно томим,
Томим и ветреным успехом,
Внимая в шуме и в тиши
Роптанье вечное души,
Зевоту подавляя смехом:
Вот как убил он восемь лет,
Утратя жизни лучший цвет.


X


В красавиц он уж не влюблялся,
А волочился как-нибудь;
Откажут – мигом утешался;
Изменят – рад был отдохнуть.
Он их искал без упоенья,
А оставлял без сожаленья,
Чуть помня их любовь и злость.
Так точно равнодушный гость
На вист вечерний приезжает,
Садится; кончилась игра:
Он уезжает со двора,
Спокойно дома засыпает
И сам не знает поутру,
Куда поедет ввечеру.


XI


Но, получив посланье Тани,
Онегин живо тронут был:
Язык девических мечтаний
В нем думы роем возмутил;
И вспомнил он Татьяны милой
И бледный цвет, и вид унылый;
И в сладостный, безгрешный сон
Душою погрузился он.
Быть может, чувствий пыл старинный
Им на минуту овладел;
Но обмануть он не хотел
Доверчивость души невинной.
Теперь мы в сад перелетим,
Где встретилась Татьяна с ним.


XII


Минуты две они молчали,
Но к ней Онегин подошел
И молвил: «Вы ко мне писали,
Не отпирайтесь. Я прочел
Души доверчивой признанья,
Любви невинной излиянья;
Мне ваша искренность мила;
Она в волненье привела
Давно умолкнувшие чувства;
Но вас хвалить я не хочу;
Я за нее вам отплачу
Признаньем также без искусства;
Примите исповедь мою:
Себя на суд вам отдаю.


XIII


Когда бы жизнь домашним кругом
Я ограничить захотел;
Когда б мне быть отцом, супругом
Приятный жребий повелел;
Когда б семейственной картиной
Пленился я хоть миг единой, —
То, верно б, кроме вас одной,
Невесты не искал иной.
Скажу без блесток мадригальных:
Нашед мой прежний идеал,
Я, верно б, вас одну избрал
В подруги дней моих печальных,
Всего прекрасного в залог,
И был бы счастлив… сколько мог!


XIV


Но я не создан для блаженства;
Ему чужда душа моя;
Напрасны ваши совершенства:
Их вовсе недостоин я.
Поверьте (совесть в том порукой),
Супружество нам будет мукой.
Я, сколько ни любил бы вас,
Привыкнув, разлюблю тотчас;
Начнете плакать: ваши слезы
Не тронут сердца моего,
А будут лишь бесить его.
Судите ж вы, какие розы
Нам заготовит Гименей
И, может быть, на много дней.


XV


Что может быть на свете хуже
Семьи, где бедная жена
Грустит о недостойном муже,
И днем и вечером одна;
Где скучный муж, ей цену зная
(Судьбу, однако ж, проклиная),
Всегда нахмурен, молчалив,
Сердит и холодно-ревнив!
Таков я. И того ль искали
Вы чистой, пламенной душой,
Когда с такою простотой,
С таким умом ко мне писали?
Ужели жребий вам такой
Назначен строгою судьбой?


XVI


Мечтам и годам нет возврата;
Не обновлю души моей…
Я вас люблю любовью брата
И, может быть, еще нежней.
Послушайте ж меня без гнева:
Сменит не раз младая дева
Мечтами легкие мечты;
Так деревцо свои листы
Меняет с каждою весною.
Так, видно, небом суждено.
Полюбите вы снова: но…
Учитесь властвовать собою:
Не всякий вас, как я, поймет;
К беде неопытность ведет».


XVII


Так проповедовал Евгений.
Сквозь слез не видя ничего,
Едва дыша, без возражений,
Татьяна слушала его.
Он подал руку ей. Печально
(Как говорится, машинально) 
Татьяна молча оперлась,
Головкой томною склонясь;
Пошли домой вкруг огорода;
Явились вместе, и никто
Не вздумал им пенять на то:
Имеет сельская свобода
Свои счастливые права,
Как и надменная Москва.


XVIII


Вы согласитесь, мой читатель,
Что очень мило поступил
С печальной Таней наш приятель;
Не в первый раз он тут явил
Души прямое благородство,
Хотя людей недоброхотство
В нем не щадило ничего:
Враги его, друзья его
(Что, может быть, одно и то же)
Его честили так и сяк.
Врагов имеет в мире всяк,
Но от друзей спаси нас, Боже!
Уж эти мне друзья, друзья!
Об них недаром вспомнил я.


XIX


А что? Да так. Я усыпляю
Пустые, черные мечты;
Я только в скобках замечаю,
Что нет презренной клеветы,
На чердаке вралем рожденной
И светской чернью ободренной,
Что нет нелепицы такой,
Ни эпиграммы площадной,
Которой бы ваш друг с улыбкой,
В кругу порядочных людей,
Без всякой злобы и затей,
Не повторил стократ ошибкой;
А впрочем, он за вас горой:
Он вас так любит… как родной!


XX


Гм! гм! Читатель благородный,
Здорова ль ваша вся родня?
Позвольте: может быть, угодно
Теперь узнать вам от меня,
Что значит именно родные. 
Родные люди вот какие:
Мы их обязаны ласкать,
Любить, душевно уважать
И, по обычаю народа,
О Рождестве их навещать
Или по почте поздравлять,
Чтоб остальное время года
Не думали о нас они…
Итак, дай Бог им долги дни!


XXI


Зато любовь красавиц нежных
Надежней дружбы и родства:
Над нею и средь бурь мятежных
Вы сохраняете права.
Конечно так. Но вихорь моды,
Но своенравие природы,
Но мненья светского поток…
А милый пол, как пух, легок.
К тому ж и мнения супруга
Для добродетельной жены
Всегда почтенны быть должны;
Так ваша верная подруга
Бывает вмиг увлечена:
Любовью шутит сатана.


XXII


Кого ж любить? Кому же верить?
Кто не изменит нам один?
Кто все дела, все речи мерит
Услужливо на наш аршин?
Кто клеветы про нас не сеет?
Кто нас заботливо лелеет?
Кому порок наш не беда?
Кто не наскучит никогда?
Призрака суетный искатель,
Трудов напрасно не губя,
Любите самого себя,
Достопочтенный мой читатель!
Предмет достойный: ничего
Любезней, верно, нет его.


XXIII


Что было следствием свиданья?
Увы, не трудно угадать!
Любви безумные страданья
Не перестали волновать
Младой души, печали жадной;
Нет, пуще страстью безотрадной
Татьяна бедная горит;
Ее постели сон бежит;
Здоровье, жизни цвет и сладость,
Улыбка, девственный покой,
Пропало всё, что звук пустой,
И меркнет милой Тани младость:
Так одевает бури тень
Едва рождающийся день.


XXIV


Увы, Татьяна увядает;
Бледнеет, гаснет и молчит!
Ничто ее не занимает,
Ее души не шевелит.
Качая важно головою,
Соседи шепчут меж собою:
Пора, пора бы замуж ей!..
Но полно. Надо мне скорей
Развеселить воображенье
Картиной счастливой любви.
Невольно, милые мои,
Меня стесняет сожаленье;
Простите мне: я так люблю
Татьяну милую мою!


XXV


Час от часу плененный боле
Красами Ольги молодой,
Владимир сладостной неволе
Предался полною душой.
Он вечно с ней. В ее покое
Они сидят в потемках двое;
Они в саду, рука с рукой,
Гуляют утренней порой;
И что ж? Любовью упоенный,
В смятенье нежного стыда,
Он только смеет иногда,
Улыбкой Ольги ободренный,
Развитым локоном играть
Иль край одежды целовать.


XXVI


Он иногда читает Оле
Нравоучительный роман,
В котором автор знает боле
Природу, чем Шатобриан,
А между тем две, три страницы
(Пустые бредни, небылицы,
Опасные для сердца дев)
Он пропускает, покраснев,
Уединясь от всех далеко,
Они над шахматной доской,
На стол облокотясь, порой
Сидят, задумавшись глубоко,
И Ленский пешкою ладью
Берет в рассеянье свою.


XXVII


Поедет ли домой: и дома
Он занят Ольгою своей.
Летучие листки альбома
Прилежно украшает ей:
То в них рисует сельски виды,
Надгробный камень, храм Киприды42
Или на лире голубка
Пером и красками слегка;
То на листках воспоминанья,
Пониже подписи других,
Он оставляет нежный стих,
Безмолвный памятник мечтанья,
Мгновенной думы долгий след,
Всё тот же после многих лет.


XXVIII


Конечно, вы не раз видали
Уездной барышни альбом,
Что все подружки измарали
С конца, с начала и кругом.
Сюда, назло правописанью,
Стихи без меры, по преданью,
В знак дружбы верной внесены,
Уменьшены, продолжены.
На первом листике встречаешь
Qu’e€crirez-vous sur ces tablettes; 
И подпись: t. a€. v. Annette; 
А на последнем прочитаешь:
«Кто любит более тебя, 
Пусть пишет далее меня». 


XXIX


Тут непременно вы найдете
Два сердца, факел и цветки;
Тут, верно, клятвы вы прочтете
В любви до гробовой доски; 
Какой-нибудь пиит армейский
Тут подмахнул стишок злодейский.
В такой альбом, мои друзья,
Признаться, рад писать и я,
Уверен будучи душою,
Что всякий мой усердный вздор
Заслужит благосклонный взор
И что потом с улыбкой злою
Не станут важно разбирать,
Остро иль нет я мог соврать.


XXX


Но вы, разрозненные томы
Из библиотеки чертей,
Великолепные альбомы,
Мученье модных рифмачей,
Вы, украшенные проворно
Толстого кистью чудотворной
Иль Баратынского пером,
Пускай сожжет вас божий гром!
Когда блистательная дама
Мне свой in-quarto подает,
И дрожь и злость меня берет,
И шевелится эпиграмма
Во глубине моей души,
А мадригалы им пиши!


XXXI


Не мадригалы Ленский пишет
В альбоме Ольги молодой;
Его перо любовью дышит,
Не хладно блещет остротой;
Что ни заметит, ни услышит
Об Ольге, он про то и пишет:
И полны истины живой
Текут элегии рекой.
Так ты, Языков вдохновенный,
В порывах сердца своего,
Поешь бог ведает кого,
И свод элегий драгоценный
Представит некогда тебе
Всю повесть о твоей судьбе.


XXXII


Но тише! Слышишь? Критик строгий
Повелевает сбросить нам
Элегии венок убогий
И нашей братье рифмачам
Кричит: «Да перестаньте плакать,
И всё одно и то же квакать,
Жалеть о прежнем, о былом: 
Довольно, пойте о другом!»
– Ты прав, и верно нам укажешь
Трубу, личину и кинжал,
И мыслей мертвый капитал
Отвсюду воскресить прикажешь:
Не так ли, друг? – Ничуть. Куда!
«Пишите оды, господа,


XXXIII


Как их писали в мощны годы,
Как было встарь заведено…»
– Одни торжественные оды!
И, полно, друг; не всё ль равно?
Припомни, что сказал сатирик!
«Чужого толка» хитрый лирик
Ужели для тебя сносней
Унылых наших рифмачей? —
«Но всё в элегии ничтожно;
Пустая цель ее жалка;
Меж тем цель оды высока
И благородна…» Тут бы можно
Поспорить нам, но я молчу:
Два века ссорить не хочу.


XXXIV


Поклонник славы и свободы,
В волненье бурных дум своих,
Владимир и писал бы оды,
Да Ольга не читала их.
Случалось ли поэтам слезным
Читать в глаза своим любезным
Свои творенья? Говорят,
Что в мире выше нет наград.
И впрямь, блажен любовник скромный,
Читающий мечты свои
Предмету песен и любви,
Красавице приятно-томной!
Блажен… хоть, может быть, она
Совсем иным развлечена.


XXXV


Но я плоды моих мечтаний
И гармонических затей
Читаю только старой няне,
Подруге юности моей,
Да после скучного обеда
Ко мне забредшего соседа,
Поймав нежданно за полу,
Душу трагедией в углу,
Или (но это кроме шуток),
Тоской и рифмами томим,
Бродя над озером моим,
Пугаю стадо диких уток:
Вняв пенью сладкозвучных строф,
Они слетают с берегов.


XXXVI. XXXVII


А что ж Онегин? Кстати, братья!
Терпенья вашего прошу:
Его вседневные занятья
Я вам подробно опишу.
Онегин жил анахоретом;
В седьмом часу вставал он летом
И отправлялся налегке
К бегущей под горой реке;
Певцу Гюльнары подражая,
Сей Геллеспонт переплывал,
Потом свой кофе выпивал,
Плохой журнал перебирая,
И одевался…


XXXVIII. XXXIX


Прогулки, чтенье, сон глубокой,
Лесная тень, журчанье струй,
Порой белянки черноокой
Младой и свежий поцелуй,
Узде послушный конь ретивый,
Обед довольно прихотливый,
Бутылка светлого вина,
Уединенье, тишина:
Вот жизнь Онегина святая;
И нечувствительно он ей
Предался, красных летних дней
В беспечной неге не считая,
Забыв и город, и друзей,
И скуку праздничных затей.


XL


Но наше северное лето,
Карикатура южных зим,
Мелькнет и нет: известно это,
Хоть мы признаться не хотим.
Уж небо осенью дышало,
Уж реже солнышко блистало,
Короче становился день,
Лесов таинственная сень
С печальным шумом обнажалась,
Ложился на поля туман,
Гусей крикливых караван
Тянулся к югу: приближалась
Довольно скучная пора;
Стоял ноябрь уж у двора.


XLI


Встает заря во мгле холодной;
На нивах шум работ умолк;
С своей волчихою голодной
Выходит на дорогу волк;
Его почуя, конь дорожный
Храпит – и путник осторожный
Несется в гору во весь дух;
На утренней заре пастух
Не гонит уж коров из хлева,
И в час полуденный в кружок
Их не зовет его рожок;
В избушке распевая, дева43
Прядет, и, зимних друг ночей,
Трещит лучинка перед ней.


XLII


И вот уже трещат морозы
И серебрятся средь полей…
(Читатель ждет уж рифмы розы; 
На, вот возьми ее скорей!)
Опрятней модного паркета
Блистает речка, льдом одета.
Мальчишек радостный народ44
Коньками звучно режет лед;
На красных лапках гусь тяжелый,
Задумав плыть по лону вод,
Ступает бережно на лед,
Скользит и падает; веселый
Мелькает, вьется первый снег,
Звездами падая на брег.


XLIII


В глуши что делать в эту пору?
Гулять? Деревня той порой
Невольно докучает взору
Однообразной наготой.
Скакать верхом в степи суровой?
Но конь, притупленной подковой
Неверный зацепляя лед,
Того и жди, что упадет.
Сиди под кровлею пустынной,
Читай: вот Прадт, вот Walter Scott.
Не хочешь? – поверяй расход,
Сердись иль пей, и вечер длинный
Кой-как пройдет, а завтра то ж,
И славно зиму проведешь.


XLIV


Прямым Онегин Чильд Гарольдом
Вдался в задумчивую лень:
Со сна садится в ванну со льдом,
И после, дома целый день,
Один, в расчеты погруженный,
Тупым кием вооруженный,
Он на бильярде в два шара
Играет с самого утра.
Настанет вечер деревенский:
Бильярд оставлен, кий забыт,
Перед камином стол накрыт,
Евгений ждет: вот едет Ленский
На тройке чалых лошадей;
Давай обедать поскорей!


XLV


Вдовы Клико или Моэта
Благословенное вино
В бутылке мерзлой для поэта
На стол тотчас принесено.
Оно сверкает Ипокреной45;
Оно своей игрой и пеной
(Подобием того-сего)
Меня пленяло: за него
Последний бедный лепт, бывало,
Давал я. Помните ль, друзья?
Его волшебная струя
Рождала глупостей не мало,
А сколько шуток и стихов,
И споров, и веселых снов!


XLVI


Но изменяет пеной шумной
Оно желудку моему,
И я Бордо благоразумный
Уж нынче предпочел ему.
К Au я больше не способен;
Au любовнице подобен
Блестящей, ветреной, живой,
И своенравной, и пустой…
Но ты, Бордо, подобен другу,
Который, в горе и в беде,
Товарищ завсегда, везде,
Готов нам оказать услугу
Иль тихий разделить досуг.
Да здравствует Бордо, наш друг!


XLVII


Огонь потух; едва золою
Подернут уголь золотой;
Едва заметною струею
Виется пар, и теплотой
Камин чуть дышит. Дым из трубок
В трубу уходит. Светлый кубок
Еще шипит среди стола.
Вечерняя находит мгла…
(Люблю я дружеские враки
И дружеский бокал вина
Порою той, что названа
Пора меж волка и собаки,
А почему, не вижу я.)
Теперь беседуют друзья:


XLVIII


«Ну, что соседки? Что Татьяна?
Что Ольга резвая твоя?»
– Налей еще мне полстакана…
Довольно, милый… Вся семья
Здорова; кланяться велели.
Ах, милый, как похорошели
У Ольги плечи, что за грудь!
Что за душа!.. Когда-нибудь
Заедем к ним; ты их обяжешь;
А то, мой друг, суди ты сам:
Два раза заглянул, а там
Уж к ним и носу не покажешь.
Да вот… какой же я болван!
Ты к ним на той неделе зван. —


XLIX


«Я?» – «Да, Татьяны именины
В субботу. Оленька и мать
Велели звать, и нет причины
Тебе на зов не приезжать». —
«Но куча будет там народу
И всякого такого сброду…» —
«И, никого, уверен я!
Кто будет там? своя семья.
Поедем, сделай одолженье!
Ну, что ж?» – «Согласен». – «Как ты мил!»
При сих словах он осушил
Стакан, соседке приношенье,
Потом разговорился вновь
Про Ольгу: такова любовь!


L


Он весел был. Чрез две недели
Назначен был счастливый срок.
И тайна брачныя постели
И сладостной любви венок
Его восторгов ожидали.
Гимена хлопоты, печали,
Зевоты хладная чреда
Ему не снились никогда.
Меж тем как мы, враги Гимена,
В домашней жизни зрим один
Ряд утомительных картин,
Роман во вкусе Лафонтена…46
Мой бедный Ленский, сердцем он
Для оной жизни был рожден.


LI


Он был любим… по крайней мере
Так думал он, и был счастлив.
Стократ блажен, кто предан вере,
Кто, хладный ум угомонив,
Покоится в сердечной неге,
Как пьяный путник на ночлеге,
Или, нежней, как мотылек,
В весенний впившийся цветок;
Но жалок тот, кто всё предвидит,
Чья не кружится голова,
Кто все движенья, все слова
В их переводе ненавидит,
Чье сердце опыт остудил
И забываться запретил!



Глава пятая

О, не знай сих страшных снов
Ты, моя Светлана!
Жуковский 



I


В тот год осенняя погода
Стояла долго на дворе,
Зимы ждала, ждала природа.
Снег выпал только в январе
На третье в ночь. Проснувшись рано,
В окно увидела Татьяна
Поутру побелевший двор,
Куртины, кровли и забор,
На стеклах легкие узоры,
Деревья в зимнем серебре,
Сорок веселых на дворе
И мягко устланные горы
Зимы блистательным ковром.
Всё ярко, всё бело кругом.


II


Зима!.. Крестьянин, торжествуя,
На дровнях обновляет путь;
Его лошадка, снег почуя,
Плетется рысью как-нибудь;
Бразды пушистые взрывая,
Летит кибитка удалая;
Ямщик сидит на облучке
В тулупе, в красном кушаке.
Вот бегает дворовый мальчик,
В салазки жучку посадив,
Себя в коня преобразив;
Шалун уж заморозил пальчик:
Ему и больно и смешно,
А мать грозит ему в окно…


III


Но, может быть, такого рода
Картины вас не привлекут:
Всё это низкая природа;
Изящного не много тут.
Согретый вдохновенья богом,
Другой поэт роскошным слогом
Живописал нам первый снег
И все оттенки зимних нег47;
Он вас пленит, я в том уверен,
Рисуя в пламенных стихах
Прогулки тайные в санях;
Но я бороться не намерен
Ни с ним покамест, ни с тобой,
Певец финляндки молодой!48


IV


Татьяна (русская душою,
Сама не зная почему)
С ее холодною красою
Любила русскую зиму,
На солнце иней в день морозный,
И сани, и зарею поздной
Сиянье розовых снегов,
И мглу крещенских вечеров.
По старине торжествовали
В их доме эти вечера:
Служанки со всего двора
Про барышень своих гадали
И им сулили каждый год
Мужьев военных и поход.


V


Татьяна верила преданьям
Простонародной старины,
И снам, и карточным гаданьям,
И предсказаниям луны.
Ее тревожили приметы;
Таинственно ей все предметы
Провозглашали что-нибудь,
Предчувствия теснили грудь.
Жеманный кот, на печке сидя,
Мурлыча, лапкой рыльце мыл:
То несомненный знак ей был,
Что едут гости. Вдруг увидя
Младой двурогий лик луны
На небе с левой стороны,


VI


Она дрожала и бледнела.
Когда ж падучая звезда
По небу темному летела
И рассыпалася, – тогда
В смятенье Таня торопилась,
Пока звезда еще катилась,
Желанье сердца ей шепнуть.
Когда случалось где-нибудь
Ей встретить черного монаха
Иль быстрый заяц меж полей
Перебегал дорогу ей,
Не зная, что начать со страха,
Предчувствий горестных полна,
Ждала несчастья уж она.


VII


Что ж? Тайну прелесть находила
И в самом ужасе она:
Так нас природа сотворила,
К противуречию склонна.
Настали святки. То-то радость!
Гадает ветреная младость,
Которой ничего не жаль,
Перед которой жизни даль
Лежит светла, необозрима;
Гадает старость сквозь очки
У гробовой своей доски,
Всё потеряв невозвратимо;
И всё равно: надежда им
Лжет детским лепетом своим.


VIII


Татьяна любопытным взором
На воск потопленный глядит:
Он чудно вылитым узором
Ей что-то чудное гласит;
Из блюда, полного водою,
Выходят кольца чередою;
И вынулось колечко ей
Под песенку старинных дней:
«Там мужички-то всё богаты, 
Гребут лопатой серебро; 
Кому поем, тому добро 
И слава!» Но сулит утраты
Сей песни жалостный напев;
Милей кошурка сердцу дев49.


IX


Морозна ночь, всё небо ясно;
Светил небесных дивный хор
Течет так тихо, так согласно…
Татьяна на широкий двор
В открытом платьице выходит,
На месяц зеркало наводит;
Но в темном зеркале одна
Дрожит печальная луна…
Чу… снег хрустит… прохожий; дева
К нему на цыпочках летит,
И голосок ее звучит
Нежней свирельного напева:
Как ваше имя? 50Смотрит он
И отвечает: Агафон.


X


Татьяна, по совету няни
Сбираясь ночью ворожить,
Тихонько приказала в бане
На два прибора стол накрыть;
Но стало страшно вдруг Татьяне…
И я – при мысли о Светлане
Мне стало страшно – так и быть…
С Татьяной нам не ворожить.
Татьяна поясок шелковый
Сняла, разделась и в постель
Легла. Над нею вьется Лель,
А под подушкою пуховой
Девичье зеркало лежит.
Утихло всё. Татьяна спит.


XI


И снится чудный сон Татьяне.
Ей снится, будто бы она
Идет по снеговой поляне,
Печальной мглой окружена;
В сугробах снежных перед нею
Шумит, клубит волной своею
Кипучий, темный и седой
Поток, не скованный зимой;
Две жердочки, склеены льдиной,
Дрожащий, гибельный мосток,
Положены через поток:
И пред шумящею пучиной,
Недоумения полна,
Остановилася она.


XII


Как на досадную разлуку,
Татьяна ропщет на ручей;
Не видит никого, кто руку
С той стороны подал бы ей;
Но вдруг сугроб зашевелился,
И кто ж из-под него явился?
Большой, взъерошенный медведь;
Татьяна ах! а он реветь,
И лапу с острыми когтями
Ей протянул; она скрепясь
Дрожащей ручкой оперлась
И боязливыми шагами
Перебралась через ручей;
Пошла – и что ж? медведь за ней!


XIII


Она, взглянуть назад не смея,
Поспешный ускоряет шаг;
Но от косматого лакея
Не может убежать никак;
Кряхтя, валит медведь несносный;
Пред ними лес; недвижны сосны
В своей нахмуренной красе;
Отягчены их ветви все
Клоками снега; сквозь вершины
Осин, берез и лип нагих
Сияет луч светил ночных;
Дороги нет; кусты, стремнины
Метелью все занесены,
Глубоко в снег погружены.


XIV


Татьяна в лес; медведь за нею;
Снег рыхлый по колено ей;
То длинный сук ее за шею
Зацепит вдруг, то из ушей
Златые серьги вырвет силой;
То в хрупком снеге с ножки милой
Увязнет мокрый башмачок;
То выронит она платок;
Поднять ей некогда; боится,
Медведя слышит за собой,
И даже трепетной рукой
Одежды край поднять стыдится;
Она бежит, он всё вослед,
И сил уже бежать ей нет.


XV


Упала в снег; медведь проворно
Ее хватает и несет;
Она бесчувственно-покорна,
Не шевельнется, не дохнет;
Он мчит ее лесной дорогой;
Вдруг меж дерев шалаш убогой;
Кругом всё глушь; отвсюду он
Пустынным снегом занесен,
И ярко светится окошко,
И в шалаше и крик, и шум;
Медведь промолвил: «Здесь мой кум:
Погрейся у него немножко!»
И в сени прямо он идет,
И на порог ее кладет.


XVI


Опомнилась, глядит Татьяна:
Медведя нет; она в сенях;
За дверью крик и звон стакана,
Как на больших похоронах;
Не видя тут ни капли толку,
Глядит она тихонько в щелку,
И что же видит?.. за столом
Сидят чудовища кругом:
Один в рогах, с собачьей мордой,
Другой с петушьей головой,
Здесь ведьма с козьей бородой,
Тут остов чопорный и гордый,
Там карла с хвостиком, а вот
Полу-журавль и полу-кот.


XVII


Еще страшней, еще чуднее:
Вот рак верхом на пауке,
Вот череп на гусиной шее
Вертится в красном колпаке,
Вот мельница вприсядку пляшет
И крыльями трещит и машет;
Лай, хохот, пенье, свист и хлоп,
Людская молвь и конский топ!51
Но что подумала Татьяна,
Когда узнала меж гостей
Того, кто мил и страшен ей,
Героя нашего романа!
Онегин за столом сидит
И в дверь украдкою глядит.


XVIII


Он знак подаст – и все хлопочут;
Он пьет – все пьют и все кричат;
Он засмеется – все хохочут;
Нахмурит брови – все молчат;
Так, он хозяин, это ясно:
И Тане уж не так ужасно,
И любопытная теперь
Немного растворила дверь…
Вдруг ветер дунул, загашая
Огонь светильников ночных;
Смутилась шайка домовых;
Онегин, взорами сверкая,
Из-за стола гремя встает;
Все встали: он к дверям идет.


XIX


И страшно ей; и торопливо
Татьяна силится бежать:
Нельзя никак; нетерпеливо
Метаясь, хочет закричать:
Не может; дверь толкнул Евгений,
И взорам адских привидений
Явилась дева; ярый смех
Раздался дико; очи всех,
Копыта, хоботы кривые,
Хвосты хохлатые, клыки,
Усы, кровавы языки,
Рога и пальцы костяные,
Всё указует на нее,
И все кричат: мое! мое!


XX


Мое! — сказал Евгений грозно,
И шайка вся сокрылась вдруг;
Осталася во тьме морозной
Младая дева с ним сам-друг;
Онегин тихо увлекает52
Татьяну в угол и слагает
Ее на шаткую скамью
И клонит голову свою
К ней на плечо; вдруг Ольга входит,
За нею Ленский; свет блеснул,
Онегин руку замахнул,
И дико он очами бродит,
И незваных гостей бранит;
Татьяна чуть жива лежит.


XXI


Спор громче, громче; вдруг Евгений
Хватает длинный нож, и вмиг
Повержен Ленский; страшно тени
Сгустились; нестерпимый крик
Раздался… хижина шатнулась…
И Таня в ужасе проснулась…
Глядит, уж в комнате светло;
В окне сквозь мерзлое стекло
Зари багряный луч играет;
Дверь отворилась. Ольга к ней,
Авроры северной алей
И легче ласточки, влетает;
«Ну, – говорит, – скажи ж ты мне,
Кого ты видела во сне?»


XXII


Но та, сестры не замечая,
В постеле с книгою лежит,
За листом лист перебирая,
И ничего не говорит.
Хоть не являла книга эта
Ни сладких вымыслов поэта,
Ни мудрых истин, ни картин,
Но ни Виргилий, ни Расин,
Ни Скотт, ни Байрон, ни Сенека,
Ни даже Дамских Мод Журнал
Так никого не занимал:
То был, друзья, Мартын Задека53,
Глава халдейских мудрецов,
Гадатель, толкователь снов.


XXIII


Сие глубокое творенье
Завез кочующий купец
Однажды к ним в уединенье
И для Татьяны наконец
Его с разрозненной Мальвиной 
Он уступил за три с полтиной,
В придачу взяв еще за них
Собранье басен площадных,
Грамматику, две Петриады,
Да Мармонтеля третий том.
Мартын Задека стал потом
Любимец Тани… Он отрады
Во всех печалях ей дарит
И безотлучно с нею спит.


XXIV


Ее тревожит сновиденье.
Не зная, как его понять,
Мечтанья страшного значенье
Татьяна хочет отыскать.
Татьяна в оглавленье кратком
Находит азбучным порядком
Слова: бор, буря, ведьма, ель,
Еж, мрак, мосток, медведь, метель
И прочая. Ее сомнений
Мартын Задека не решит;
Но сон зловещий ей сулит
Печальных много приключений.
Дней несколько она потом
Всё беспокоилась о том.


XXV


Но вот багряною рукою54
Заря от утренних долин
Выводит с солнцем за собою
Веселый праздник именин.
С утра дом Лариной гостями
Весь полон; целыми семьями
Соседи съехались в возках,
В кибитках, в бричках и в санях.
В передней толкотня, тревога;
В гостиной встреча новых лиц,
Лай мосек, чмоканье девиц,
Шум, хохот, давка у порога,
Поклоны, шарканье гостей,
Кормилиц крик и плач детей.


XXVI


С своей супругою дородной
Приехал толстый Пустяков;
Гвоздин, хозяин превосходный,
Владелец нищих мужиков;
Скотинины, чета седая,
С детьми всех возрастов, считая
От тридцати до двух годов;
Уездный франтик Петушков,
Мой брат двоюродный, Буянов,
В пуху, в картузе с козырьком55
(Как вам, конечно, он знаком),
И отставной советник Флянов,
Тяжелый сплетник, старый плут,
Обжора, взяточник и шут.


XXVII


С семьей Панфила Харликова
Приехал и мосье Трике,
Остряк, недавно из Тамбова,
В очках и в рыжем парике.
Как истинный француз, в кармане
Трике привез куплет Татьяне
На голос, знаемый детьми:
Re€veillez-vous, belle endormie 56. 
Меж ветхих песен альманаха
Был напечатан сей куплет;
Трике, догадливый поэт,
Его на свет явил из праха,
И смело вместо belle Nina 57
Поставил belle Tatiana 58. 


XXVIII


И вот из ближнего посада,
Созревших барышень кумир,
Уездных матушек отрада,
Приехал ротный командир;
Вошел… Ах, новость, да какая!
Музыка будет полковая!
Полковник сам ее послал.
Какая радость: будет бал!
Девчонки прыгают заране59;
Но кушать подали. Четой
Идут за стол рука с рукой.
Теснятся барышни к Татьяне;
Мужчины против; и, крестясь,
Толпа жужжит, за стол садясь.


XXIX


На миг умолкли разговоры;
Уста жуют. Со всех сторон
Гремят тарелки и приборы,
Да рюмок раздается звон.
Но вскоре гости понемногу
Подъемлют общую тревогу.
Никто не слушает, кричат,
Смеются, спорят и пищат.
Вдруг двери настежь. Ленский входит,
И с ним Онегин. «Ах, творец! —
Кричит хозяйка: – наконец!»
Теснятся гости, всяк отводит
Приборы, стулья поскорей;
Зовут, сажают двух друзей.


XXX


Сажают прямо против Тани,
И, утренней луны бледней
И трепетней гонимой лани,
Она темнеющих очей
Не подымает: пышет бурно
В ней страстный жар; ей душно, дурно;
Она приветствий двух друзей
Не слышит, слезы из очей
Хотят уж капать; уж готова
Бедняжка в обморок упасть;
Но воля и рассудка власть
Превозмогли. Она два слова
Сквозь зубы молвила тишком
И усидела за столом.


XXXI


Траги-нервических явлений,
Девичьих обмороков, слез
Давно терпеть не мог Евгений:
Довольно их он перенес.
Чудак, попав на пир огромный,
Уж был сердит. Но, девы томной
Заметя трепетный порыв,
С досады взоры опустив,
Надулся он и, негодуя,
Поклялся Ленского взбесить
И уж порядком отомстить.
Теперь, заране торжествуя,
Он стал чертить в душе своей
Карикатуры всех гостей.


XXXII


Конечно, не один Евгений
Смятенье Тани видеть мог;
Но целью взоров и суждений
В то время жирный был пирог
(К несчастию, пересоленный);
Да вот в бутылке засмоленной,
Между жарким и блан-манже,
Цимлянское несут уже;
За ним строй рюмок узких, длинных,
Подобно талии твоей,
Зизи, кристалл души моей,
Предмет стихов моих невинных,
Любви приманчивый фиал,
Ты, от кого я пьян бывал!


XXXIII


Освободясь от пробки влажной,
Бутылка хлопнула; вино
Шипит; и вот с осанкой важной,
Куплетом мучимый давно,
Трике встает; пред ним собранье
Хранит глубокое молчанье.
Татьяна чуть жива; Трике,
К ней обратясь с листком в руке,
Запел, фальшивя. Плески, клики
Его приветствуют. Она
Певцу присесть принуждена;
Поэт же скромный, хоть великий,
Ее здоровье первый пьет
И ей куплет передает.


XXXIV


Пошли приветы, поздравленья:
Татьяна всех благодарит.
Когда же дело до Евгенья
Дошло, то девы томный вид,
Ее смущение, усталость
В его душе родили жалость:
Он молча поклонился ей;
Но как-то взор его очей
Был чудно нежен. Оттого ли,
Что он и вправду тронут был,
Иль он, кокетствуя, шалил,
Невольно ль, иль из доброй воли,
Но взор сей нежность изъявил:
Он сердце Тани оживил.


XXXV


Гремят отдвинутые стулья;
Толпа в гостиную валит:
Так пчел из лакомого улья
На ниву шумный рой летит.
Довольный праздничным обедом
Сосед сопит перед соседом;
Подсели дамы к камельку;
Девицы шепчут в уголку;
Столы зеленые раскрыты:
Зовут задорных игроков
Бостон и ломбер стариков,
И вист, доныне знаменитый,
Однообразная семья,
Все жадной скуки сыновья.


XXXVI


Уж восемь робертов сыграли
Герои виста; восемь раз
Они места переменяли;
И чай несут. Люблю я час
Определять обедом, чаем
И ужином. Мы время знаем
В деревне без больших сует:
Желудок – верный наш брегет;
И кстати я замечу в скобках,
Что речь веду в моих строфах
Я столь же часто о пирах,
О разных кушаньях и пробках,
Как ты, божественный Омир,
Ты, тридцати веков кумир!


XXXVII. XXXVIII. XXXIX


Но чай несут: девицы чинно
Едва за блюдечки взялись,
Вдруг из-за двери в зале длинной
Фагот и флейта раздались.
Обрадован музыки громом,
Оставя чашку чаю с ромом,
Парис окружных городков,
Подходит к Ольге Петушков,
К Татьяне Ленский; Харликову,
Невесту переспелых лет,
Берет тамбовский мой поэт,
Умчал Буянов Пустякову,
И в залу высыпали все,
И бал блестит во всей красе.


XL


В начале моего романа
(Смотрите первую тетрадь)
Хотелось вроде мне Альбана
Бал петербургский описать;
Но, развлечен пустым мечтаньем,
Я занялся воспоминаньем
О ножках мне знакомых дам.
По вашим узеньким следам,
О ножки, полно заблуждаться!
С изменой юности моей
Пора мне сделаться умней,
В делах и в слоге поправляться,
И эту пятую тетрадь
От отступлений очищать.


XLI


Однообразный и безумный,
Как вихорь жизни молодой,
Кружится вальса вихорь шумный;
Чета мелькает за четой.
К минуте мщенья приближаясь,
Онегин, втайне усмехаясь,
Подходит к Ольге. Быстро с ней
Вертится около гостей,
Потом на стул ее сажает,
Заводит речь о том, о сем;
Спустя минуты две потом
Вновь с нею вальс он продолжает;
Все в изумленье. Ленский сам
Не верит собственным глазам.


XLII


Мазурка раздалась. Бывало,
Когда гремел мазурки гром,
В огромной зале всё дрожало,
Паркет трещал под каблуком,
Тряслися, дребезжали рамы;
Теперь не то: и мы, как дамы,
Скользим по лаковым доскам.
Но в городах, по деревням
Еще мазурка сохранила
Первоначальные красы:
Припрыжки, каблуки, усы
Всё те же; их не изменила
Лихая мода, наш тиран,
Недуг новейших россиян.


XLIII. XLIV


Буянов, братец мой задорный,
К герою нашему подвел
Татьяну с Ольгою; проворно
Онегин с Ольгою пошел;
Ведет ее, скользя небрежно,
И, наклонясь, ей шепчет нежно
Какой-то пошлый мадригал
И руку жмет – и запылал
В ее лице самолюбивом
Румянец ярче. Ленский мой
Всё видел: вспыхнул, сам не свой;
В негодовании ревнивом
Поэт конца мазурки ждет
И в котильон ее зовет.


XLV


Но ей нельзя. Нельзя? Но что же?
Да Ольга слово уж дала
Онегину. О Боже, Боже!
Что слышит он? Она могла…
Возможно ль? Чуть лишь из пеленок,
Кокетка, ветреный ребенок!
Уж хитрость ведает она,
Уж изменять научена!
Не в силах Ленский снесть удара;
Проказы женские кляня,
Выходит, требует коня
И скачет. Пистолетов пара,
Две пули – больше ничего —
Вдруг разрешат судьбу его.



Глава шестая

La, sotto i giorni nubilosi e brevi,
Nasce una gente a cuil morir non dole.
Petr. 60



I


Заметив, что Владимир скрылся,
Онегин, скукой вновь гоним,
Близ Ольги в думу погрузился,
Довольный мщением своим.
За ним и Оленька зевала,
Глазами Ленского искала,
И бесконечный котильон
Ее томил, как тяжкий сон.
Но кончен он. Идут за ужин.
Постели стелют; для гостей
Ночлег отводят от сеней
До самой девичьи. Всем нужен
Покойный сон. Онегин мой
Один уехал спать домой.


II


Всё успокоилось: в гостиной
Храпит тяжелый Пустяков
С своей тяжелой половиной.
Гвоздин, Буянов, Петушков
И Флянов, не совсем здоровый,
На стульях улеглись в столовой,
А на полу мосье Трике,
В фуфайке, в старом колпаке.
Девицы в комнатах Татьяны
И Ольги все объяты сном.
Одна, печальна под окном
Озарена лучом Дианы,
Татьяна бедная не спит
И в поле темное глядит.


III


Его нежданным появленьем,
Мгновенной нежностью очей
И странным с Ольгой поведеньем
До глубины души своей
Она проникнута; не может
Никак понять его; тревожит
Ее ревнивая тоска,
Как будто хладная рука
Ей сердце жмет, как будто бездна
Под ней чернеет и шумит…
«Погибну, – Таня говорит, —
Но гибель от него любезна.
Я не ропщу: зачем роптать?
Не может он мне счастья дать».


IV


Вперед, вперед, моя исторья!
Лицо нас новое зовет.
В пяти верстах от Красногорья,
Деревни Ленского, живет
И здравствует еще доныне
В философической пустыне
Зарецкий, некогда буян,
Картежной шайки атаман,
Глава повес, трибун трактирный,
Теперь же добрый и простой
Отец семейства холостой,
Надежный друг, помещик мирный
И даже честный человек:
Так исправляется наш век!


V


Бывало, льстивый голос света
В нем злую храбрость выхвалял:
Он, правда, в туз из пистолета
В пяти саженях попадал,
И то сказать, что и в сраженье
Раз в настоящем упоенье
Он отличился, смело в грязь
С коня калмыцкого свалясь,
Как зюзя пьяный, и французам
Достался в плен: драгой залог!
Новейший Регул, чести бог,
Готовый вновь предаться узам,
Чтоб каждым утром у Вери61
В долг осушать бутылки три.


VI


Бывало, он трунил забавно,
Умел морочить дурака
И умного дурачить славно,
Иль явно, иль исподтишка,
Хоть и ему иные штуки
Не проходили без науки,
Хоть иногда и сам впросак
Он попадался, как простак.
Умел он весело поспорить,
Остро и тупо отвечать,
Порой расчетливо смолчать,
Порой расчетливо повздорить,
Друзей поссорить молодых
И на барьер поставить их,


VII


Иль помириться их заставить,
Дабы позавтракать втроем,
И после тайно обесславить
Веселой шуткою, враньем.
Sel alia tempora!62Удалость
(Как сон любви, другая шалость)
Проходит с юностью живой.
Как я сказал, Зарецкий мой,
Под сень черемух и акаций
От бурь укрывшись наконец,
Живет, как истинный мудрец,
Капусту садит, как Гораций,
Разводит уток и гусей
И учит азбуке детей.


VIII


Он был не глуп; и мой Евгений,
Не уважая сердца в нем,
Любил и дух его суждений,
И здравый толк о том, о сем.
Он с удовольствием, бывало,
Видался с ним, и так нимало
Поутру не был удивлен,
Когда его увидел он.
Тот после первого привета,
Прервав начатый разговор,
Онегину, осклабя взор,
Вручил записку от поэта.
К окну Онегин подошел
И про себя ее прочел.


IX


То был приятный, благородный,
Короткий вызов, иль картель: 
Учтиво, с ясностью холодной
Звал друга Ленский на дуэль.
Онегин с первого движенья,
К послу такого порученья
Оборотясь, без лишних слов
Сказал, что он всегда готов. 
Зарецкий встал без объяснений;
Остаться доле не хотел,
Имея дома много дел,
И тотчас вышел; но Евгений
Наедине с своей душой
Был недоволен сам собой.


X


И поделом: в разборе строгом,
На тайный суд себя призвав,
Он обвинял себя во многом:
Во-первых, он уж был неправ,
Что над любовью робкой, нежной
Так подшутил вечор небрежно.
А во-вторых: пускай поэт
Дурачится; в осьмнадцать лет
Оно простительно. Евгений,
Всем сердцем юношу любя,
Был должен оказать себя
Не мячиком предрассуждений,
Не пылким мальчиком, бойцом,
Но мужем с честью и с умом.


XI


Он мог бы чувства обнаружить,
А не щетиниться, как зверь;
Он должен был обезоружить
Младое сердце. «Но теперь
Уж поздно; время улетело…
К тому ж – он мыслит – в это дело
Вмешался старый дуэлист;
Он зол, он сплетник, он речист…
Конечно, быть должно презренье
Ценой его забавных слов,
Но шепот, хохотня глупцов…»
И вот общественное мненье!63
Пружина чести, наш кумир!
И вот на чем вертится мир!


XII


Кипя враждой нетерпеливой,
Ответа дома ждет поэт;
И вот сосед велеречивый
Привез торжественно ответ.
Теперь ревнивцу то-то праздник!
Он всё боялся, чтоб проказник
Не отшутился как-нибудь,
Уловку выдумав и грудь
Отворотив от пистолета.
Теперь сомненья решены:
Они на мельницу должны
Приехать завтра до рассвета,
Взвести друг на друга курок
И метить в ляжку иль в висок.


XIII


Решась кокетку ненавидеть,
Кипящий Ленский не хотел
Пред поединком Ольгу видеть,
На солнце, на часы смотрел,
Махнул рукою напоследок —
И очутился у соседок.
Он думал Оленьку смутить,
Своим приездом поразить;
Не тут-то было: как и прежде,
На встречу бедного певца
Прыгнула Оленька с крыльца,
Подобна ветреной надежде,
Резва, беспечна, весела,
Ну точно та же, как была.


XIV


«Зачем вечор так рано скрылись?» —
Был первый Оленькин вопрос.
Все чувства в Ленском помутились,
И молча он повесил нос.
Исчезла ревность и досада
Пред этой ясностию взгляда,
Пред этой нежной простотой,
Пред этой резвою душой!..
Он смотрит в сладком умиленье;
Он видит: он еще любим;
Уж он, раскаяньем томим,
Готов просить у ней прощенье,
Трепещет, не находит слов,
Он счастлив, он почти здоров…


XV. XVI. XVII


И вновь задумчивый, унылый
Пред милой Ольгою своей,
Владимир не имеет силы
Вчерашний день напомнить ей;
Он мыслит: «Буду ей спаситель.
Не потерплю, чтоб развратитель
Огнем и вздохов и похвал
Младое сердце искушал;
Чтоб червь презренный, ядовитый
Точил лилеи стебелек;
Чтобы двухутренний цветок
Увял еще полураскрытый».
Всё это значило, друзья:
С приятелем стреляюсь я.


XVIII


Когда б он знал, какая рана
Моей Татьяны сердце жгла!
Когда бы ведала Татьяна,
Когда бы знать она могла,
Что завтра Ленский и Евгений
Заспорят о могильной сени;
Ах, может быть, ее любовь
Друзей соединила б вновь!
Но этой страсти и случайно
Еще никто не открывал.
Онегин обо всем молчал;
Татьяна изнывала тайно;
Одна бы няня знать могла,
Да недогадлива была.


XIX


Весь вечер Ленский был рассеян,
То молчалив, то весел вновь;
Но тот, кто музою взлелеян,
Всегда таков: нахмуря бровь,
Садился он за клавикорды
И брал на них одни аккорды,
То, к Ольге взоры устремив,
Шептал: не правда ль? я счастлив.
Но поздно; время ехать. Сжалось
В нем сердце, полное тоской;
Прощаясь с девой молодой,
Оно как будто разрывалось.
Она глядит ему в лицо.
«Что с вами?» – «Так». – И на крыльцо.


XX


Домой приехав, пистолеты
Он осмотрел, потом вложил
Опять их в ящик и, раздетый,
При свечке, Шиллера открыл;
Но мысль одна его объемлет;
В нем сердце грустное не дремлет:
С неизъяснимою красой
Он видит Ольгу пред собой.
Владимир книгу закрывает,
Берет перо; его стихи,
Полны любовной чепухи,
Звучат и льются. Их читает
Он вслух, в лирическом жару,
Как Дельвиг пьяный на пиру.


XXI


Стихи на случай сохранились;
Я их имею; вот они:
«Куда, куда вы удалились,
Весны моей златые дни?
Что день грядущий мне готовит?
Его мой взор напрасно ловит,
В глубокой мгле таится он.
Нет нужды; прав судьбы закон.
Паду ли я, стрелой пронзенный,
Иль мимо пролетит она,
Всё благо: бдения и сна
Приходит час определенный;
Благословен и день забот,
Благословен и тьмы приход!


XXII


Блеснет заутра луч денницы
И заиграет яркий день;
А я, быть может, я гробницы
Сойду в таинственную сень,
И память юного поэта
Поглотит медленная Лета,
Забудет мир меня; но ты
Придешь ли, дева красоты,
Слезу пролить над ранней урной
И думать: он меня любил,
Он мне единой посвятил
Рассвет печальный жизни бурной!..
Сердечный друг, желанный друг,
Приди, приди: я твой супруг!..»


XXIII


Так он писал темно и вяло 
(Что романтизмом мы зовем,
Хоть романтизма тут нимало
Не вижу я; да что нам в том?)
И наконец перед зарею,
Склонясь усталой головою,
На модном слове идеал 
Тихонько Ленский задремал;
Но только сонным обаяньем
Он позабылся, уж сосед
В безмолвный входит кабинет
И будит Ленского воззваньем:
«Пора вставать: седьмой уж час.
Онегин, верно, ждет уж нас».


XXIV


Но ошибался он: Евгений
Спал в это время мертвым сном.
Уже редеют ночи тени
И встречен Веспер петухом;
Онегин спит себе глубоко.
Уж солнце катится высоко,
И перелетная метель
Блестит и вьется; но постель
Еще Евгений не покинул,
Еще над ним летает сон.
Вот наконец проснулся он
И полы завеса раздвинул;
Глядит – и видит, что пора
Давно уж ехать со двора.


XXV


Он поскорей звонит. Вбегает
К нему слуга француз Гильо,
Халат и туфли предлагает
И подает ему белье.
Спешит Онегин одеваться,
Слуге велит приготовляться
С ним вместе ехать и с собой
Взять также ящик боевой.
Готовы санки беговые.
Он сел, на мельницу летит.
Примчались. Он слуге велит
Лепажа 64стволы роковые
Нести за ним, а лошадям
Отъехать в поле к двум дубкам.


XXVI


Опершись на плотину, Ленский
Давно нетерпеливо ждал;
Меж тем, механик деревенский,
Зарецкий жернов осуждал.
Идет Онегин с извиненьем.
«Но где же, – молвил с изумленьем
Зарецкий, – где ваш секундант?»
В дуэлях классик и педант,
Любил методу он из чувства,
И человека растянуть
Он позволял – не как-нибудь,
Но в строгих правилах искусства,
По всем преданьям старины
(Что похвалить мы в нем должны).


XXVII


«Мой секундант? – сказал Евгений, —
Вот он: мой друг, monsieur Guillot.
Я не предвижу возражений
На представление мое;
Хоть человек он неизвестный,
Но уж, конечно, малый честный».
Зарецкий губу закусил.
Онегин Ленского спросил:
«Что ж, начинать?» – «Начнем, пожалуй»,—
Сказал Владимир. И пошли
За мельницу. Пока вдали
Зарецкий наш и честный малый 
Вступили в важный договор,
Враги стоят, потупя взор.


XXVIII


Враги! Давно ли друг от друга
Их жажда крови отвела?
Давно ль они часы досуга,
Трапезу, мысли и дела
Делили дружно? Ныне злобно,
Врагам наследственным подобно,
Как в страшном, непонятном сне,
Они друг другу в тишине
Готовят гибель хладнокровно…
Не засмеяться ль им, пока
Не обагрилась их рука,
Не разойтиться ль полюбовно?..
Но дико светская вражда
Боится ложного стыда.


XXIX


Вот пистолеты уж блеснули,
Гремит о шомпол молоток.
В граненый ствол уходят пули,
И щелкнул в первый раз курок.
Вот порох струйкой сероватой
На полку сыплется. Зубчатый,
Надежно ввинченный кремень
Взведен еще. За ближний пень
Становится Гильо смущенный.
Плащи бросают два врага.
Зарецкий тридцать два шага
Отмерил с точностью отменной,
Друзей развел по крайний след,
И каждый взял свой пистолет.


XXX


«Теперь сходитесь».
Хладнокровно,
Еще не целя, два врага
Походкой твердой, тихо, ровно
Четыре перешли шага,
Четыре смертные ступени.
Свой пистолет тогда Евгений,
Не преставая наступать,
Стал первый тихо подымать.
Вот пять шагов еще ступили,
И Ленский, жмуря левый глаз,
Стал также целить – но как раз
Онегин выстрелил… Пробили
Часы урочные: поэт
Роняет молча пистолет.


XXXI


На грудь кладет тихонько руку
И падает. Туманный взор
Изображает смерть, не муку.
Так медленно по скату гор,
На солнце искрами блистая,
Спадает глыба снеговая.
Мгновенным холодом облит,
Онегин к юноше спешит,
Глядит, зовет его… напрасно:
Его уж нет. Младой певец
Нашел безвременный конец!
Дохнула буря, цвет прекрасный
Увял на утренней заре,
Потух огонь на алтаре!..


XXXII


Недвижим он лежал, и странен
Был томный мир его чела.
Под грудь он был навылет ранен;
Дымясь, из раны кровь текла.
Тому назад одно мгновенье
В сем сердце билось вдохновенье,
Вражда, надежда и любовь,
Играла жизнь, кипела кровь;
Теперь, как в доме опустелом,
Всё в нем и тихо и темно;
Замолкло навсегда оно.
Закрыты ставни, окна мелом
Забелены. Хозяйки нет.
А где, Бог весть. Пропал и след.


XXXIII


Приятно дерзкой эпиграммой
Взбесить оплошного врага;
Приятно зреть, как он, упрямо
Склонив бодливые рога,
Невольно в зеркало глядится
И узнавать себя стыдится;
Приятней, если он, друзья,
Завоет сдуру: это я!
Еще приятнее в молчанье
Ему готовить честный гроб
И тихо целить в бледный лоб
На благородном расстоянье;
Но отослать его к отцам
Едва ль приятно будет вам.


XXXIV


Что ж, если вашим пистолетом
Сражен приятель молодой,
Нескромным взглядом, иль ответом,
Или безделицей иной
Вас оскорбивший за бутылкой,
Иль даже сам в досаде пылкой
Вас гордо вызвавший на бой,
Скажите: вашею душой
Какое чувство овладеет,
Когда недвижим, на земле
Пред вами с смертью на челе,
Он постепенно костенеет,
Когда он глух и молчалив
На ваш отчаянный призыв?


XXXV


В тоске сердечных угрызений,
Рукою стиснув пистолет,
Глядит на Ленского Евгений.
«Ну, что ж? убит», – решил сосед.
Убит!.. Сим страшным восклицаньем
Сражен, Онегин с содроганьем
Отходит и людей зовет.
Зарецкий бережно кладет
На сани труп оледенелый;
Домой везет он страшный клад.
Почуя мертвого, храпят
И бьются кони, пеной белой
Стальные мочат удила,
И полетели как стрела.


XXXVI


Друзья мои, вам жаль поэта:
Во цвете радостных надежд,
Их не свершив еще для света,
Чуть из младенческих одежд,
Увял! Где жаркое волненье,
Где благородное стремленье
И чувств и мыслей молодых,
Высоких, нежных, удалых?
Где бурные любви желанья,
И жажда знаний и труда,
И страх порока и стыда,
И вы, заветные мечтанья,
Вы, призрак жизни неземной,
Вы, сны поэзии святой!


XXXVII


Быть может, он для блага мира
Иль хоть для славы был рожден;
Его умолкнувшая лира
Гремучий, непрерывный звон
В веках поднять могла. Поэта,
Быть может, на ступенях света
Ждала высокая ступень.
Его страдальческая тень,
Быть может, унесла с собою
Святую тайну, и для нас
Погиб животворящий глас,
И за могильною чертою
К ней не домчится гимн времен,
Благословение племен.


XXXVIII. XXXIX


А может быть и то: поэта
Обыкновенный ждал удел.
Прошли бы юношества лета:
В нем пыл души бы охладел.
Во многом он бы изменился,
Расстался б с музами, женился,
В деревне, счастлив и рогат,
Носил бы стеганый халат;
Узнал бы жизнь на самом деле,
Подагру б в сорок лет имел,
Пил, ел, скучал, толстел, хирел.
И наконец в своей постеле
Скончался б посреди детей,
Плаксивых баб и лекарей.


XL


Но что бы ни было, читатель,
Увы! любовник молодой,
Поэт, задумчивый мечтатель,
Убит приятельской рукой!
Есть место: влево от селенья,
Где жил питомец вдохновенья,
Две сосны корнями срослись;
Под ними струйки извились
Ручья соседственной долины.
Там пахарь любит отдыхать,
И жницы в волны погружать
Приходят звонкие кувшины;
Там у ручья в тени густой
Поставлен памятник простой.


XLI


Под ним (как начинает капать
Весенний дождь на злак полей)
Пастух, плетя свой пестрый лапоть,
Поет про волжских рыбарей;
И горожанка молодая,
В деревне лето провождая,
Когда стремглав верхом она
Несется по полям одна,
Коня пред ним остановляет,
Ремянный повод натянув,
И, флер от шляпы отвернув,
Глазами беглыми читает
Простую надпись – и слеза
Туманит нежные глаза.


XLII


И шагом едет в чистом поле,
В мечтанья погрузясь, она;
Душа в ней долго поневоле
Судьбою Ленского полна;
И мыслит: «Что-то с Ольгой стало?
В ней сердце долго ли страдало,
Иль скоро слез прошла пора?
И где теперь ее сестра?
И где ж беглец людей и света,
Красавиц модных модный враг,
Где этот пасмурный чудак,
Убийца юного поэта?»
Со временем отчет я вам
Подробно обо всем отдам,


XLIII


Но не теперь. Хоть я сердечно
Люблю героя моего,
Хоть возвращусь к нему, конечно,
Но мне теперь не до него.
Лета к суровой прозе клонят,
Лета шалунью рифму гонят,
И я – со вздохом признаюсь —
За ней ленивей волочусь.
Перу старинной нет охоты
Марать летучие листы;
Другие, хладные мечты,
Другие, строгие заботы
И в шуме света и в тиши
Тревожат сон моей души.


XLIV


Познал я глас иных желаний,
Познал я новую печаль;
Для первых нет мне упований,
А старой мне печали жаль.
Мечты, мечты! где ваша сладость?
Где, вечная к ней рифма, младость? 
Ужель и вправду наконец
Увял, увял ее венец?
Ужель и впрямь и в самом деле
Без элегических затей
Весна моих промчалась дней
(Что я шутя твердил доселе)?
И ей ужель возврата нет?
Ужель мне скоро тридцать лет?


XLV


Так, полдень мой настал, и нужно
Мне в том сознаться, вижу я.
Но так и быть: простимся дружно,
О юность легкая моя!
Благодарю за наслажденья,
За грусть, за милые мученья,
За шум, за бури, за пиры,
За все, за все твои дары;
Благодарю тебя. Тобою,
Среди тревог и в тишине,
Я насладился… и вполне;
Довольно! С ясною душою
Пускаюсь ныне в новый путь
От жизни прошлой отдохнуть.


XLVI


Дай оглянусь. Простите ж, сени,
Где дни мои текли в глуши,
Исполнены страстей и лени
И снов задумчивой души.
А ты, младое вдохновенье,
Волнуй мое воображенье,
Дремоту сердца оживляй,
В мой угол чаще прилетай,
Не дай остыть душе поэта,
Ожесточиться, очерстветь
И наконец окаменеть
В мертвящем упоенье света,
В сем омуте, где с вами я
Купаюсь, милые друзья!65



Глава седьмая

Москва, России дочь любима,
Где равную тебе сыскать?
Дмитриев 

Как не любить родной Москвы?
Баратынский 

Гоненье на Москву! что значит видеть свет!
Где ж лучше?
Где нас нет.
Грибоедов 



I


Гонимы вешними лучами,
С окрестных гор уже снега
Сбежали мутными ручьями
На потопленные луга.
Улыбкой ясною природа
Сквозь сон встречает утро года;
Синея блещут небеса.
Еще прозрачные леса
Как будто пухом зеленеют.
Пчела за данью полевой
Летит из кельи восковой.
Долины сохнут и пестреют;
Стада шумят, и соловей
Уж пел в безмолвии ночей.


II


Как грустно мне твое явленье,
Весна, весна! пора любви!
Какое томное волненье
В моей душе, в моей крови!
С каким тяжелым умиленьем
Я наслаждаюсь дуновеньем
В лицо мне веющей весны
На лоне сельской тишины!
Или мне чуждо наслажденье,
И всё, что радует, живит,
Всё, что ликует и блестит,
Наводит скуку и томленье
На душу мертвую давно,
И всё ей кажется темно?


III


Или, не радуясь возврату
Погибших осенью листов,
Мы помним горькую утрату,
Внимая новый шум лесов;
Или с природой оживленной
Сближаем думою смущенной
Мы увяданье наших лет,
Которым возрожденья нет?
Быть может, в мысли нам приходит
Средь поэтического сна
Иная, старая весна
И в трепет сердце нам приводит
Мечтой о дальней стороне,
О чудной ночи, о луне…


IV


Вот время: добрые ленивцы,
Эпикурейцы-мудрецы,
Вы, равнодушные счастливцы,
Вы, школы Левшина66птенцы,
Вы, деревенские Приамы,
И вы, чувствительные дамы,
Весна в деревню вас зовет,
Пора тепла, цветов, работ,
Пора гуляний вдохновенных
И соблазнительных ночей.
В поля, друзья! скорей, скорей,
В каретах, тяжко нагруженных,
На долгих иль на почтовых
Тянитесь из застав градских.


V


И вы, читатель благосклонный,
В своей коляске выписной
Оставьте град неугомонный,
Где веселились вы зимой;
С моею музой своенравной
Пойдемте слушать шум дубравный
Над безыменною рекой
В деревне, где Евгений мой,
Отшельник праздный и унылый,
Еще недавно жил зимой
В соседстве Тани молодой,
Моей мечтательницы милой,
Но где его теперь уж нет…
Где грустный он оставил след.


VI


Меж гор, лежащих полукругом,
Пойдем туда, где ручеек,
Виясь, бежит зеленым лугом
К реке сквозь липовый лесок.
Там соловей, весны любовник,
Всю ночь поет; цветет шиповник,
И слышен говор ключевой, —
Там виден камень гробовой
В тени двух сосен устарелых.
Пришельцу надпись говорит:
«Владимир Ленской здесь лежит,
Погибший рано смертью смелых,
В такой-то год, таких-то лет.
Покойся, юноша-поэт!»


VII


На ветви сосны преклоненной,
Бывало, ранний ветерок
Над этой урною смиренной
Качал таинственный венок.
Бывало, в поздние досуги
Сюда ходили две подруги,
И на могиле при луне,
Обнявшись, плакали оне.
Но ныне… памятник унылый
Забыт. К нему привычный след
Заглох. Венка на ветви нет;
Один под ним, седой и хилый,
Пастух по-прежнему поет
И обувь бедную плетет.


VIII. IX. X


Мой бедный Ленский! изнывая,
Не долго плакала она.
Увы! невеста молодая
Своей печали неверна.
Другой увлек ее вниманье,
Другой успел ее страданье
Любовной лестью усыпить,
Улан умел ее пленить,
Улан любим ее душою…
И вот уж с ним пред алтарем
Она стыдливо под венцом
Стоит с поникшей головою,
С огнем в потупленных очах,
С улыбкой легкой на устах.


XI


Мой бедный Ленский! за могилой
В пределах вечности глухой
Смутился ли, певец унылый,
Измены вестью роковой,
Или над Летой усыпленный
Поэт, бесчувствием блаженный,
Уж не смущается ничем,
И мир ему закрыт и нем?..
Так! равнодушное забвенье
За гробом ожидает нас.
Врагов, друзей, любовниц глас
Вдруг молкнет. Про одно именье
Наследников сердитый хор
Заводит непристойный спор.


XII


И скоро звонкий голос Оли
В семействе Лариных умолк.
Улан, своей невольник доли,
Был должен ехать с нею в полк.
Слезами горько обливаясь,
Старушка, с дочерью прощаясь,
Казалось, чуть жива была,
Но Таня плакать не могла;
Лишь смертной бледностью покрылось
Ее печальное лицо.
Когда все вышли на крыльцо,
И всё, прощаясь, суетилось
Вокруг кареты молодых,
Татьяна проводила их.


XIII


И долго, будто сквозь тумана,
Она глядела им вослед…
И вот одна, одна Татьяна!
Увы! подруга стольких лет,
Ее голубка молодая,
Ее наперсница родная,
Судьбою вдаль занесена,
С ней навсегда разлучена.
Как тень она без цели бродит,
То смотрит в опустелый сад…
Нигде, ни в чем ей нет отрад,
И облегченья не находит
Она подавленным слезам,
И сердце рвется пополам.


XIV


И в одиночестве жестоком
Сильнее страсть ее горит,
И об Онегине далеком
Ей сердце громче говорит.
Она его не будет видеть;
Она должна в нем ненавидеть
Убийцу брата своего;
Поэт погиб… но уж его
Никто не помнит, уж другому
Его невеста отдалась.
Поэта память пронеслась,
Как дым по небу голубому,
О нем два сердца, может быть,
Еще грустят… На что грустить?..


XV


Был вечер. Небо меркло. Воды
Струились тихо. Жук жужжал.
Уж расходились хороводы;
Уж за рекой, дымясь, пылал
Огонь рыбачий. В поле чистом,
Луны при свете серебристом
В свои мечты погружена,
Татьяна долго шла одна.
Шла, шла. И вдруг перед собою
С холма господский видит дом,
Селенье, рощу под холмом
И сад над светлою рекою.
Она глядит – и сердце в ней
Забилось чаще и сильней.


XVI


Ее сомнения смущают:
«Пойду ль вперед, пойду ль назад?..
Его здесь нет. Меня не знают…
Взгляну на дом, на этот сад».
И вот с холма Татьяна сходит,
Едва дыша; кругом обводит
Недоуменья полный взор…
И входит на пустынный двор.
К ней, лая, кинулись собаки.
На крик испуганный ея
Ребят дворовая семья
Сбежалась шумно. Не без драки
Мальчишки разогнали псов,
Взяв барышню под свой покров.


XVII


«Увидеть барский дом нельзя ли?» —
Спросила Таня. Поскорей
К Анисье дети побежали
У ней ключи взять от сеней;
Анисья тотчас к ней явилась,
И дверь пред ними отворилась,
И Таня входит в дом пустой,
Где жил недавно наш герой.
Она глядит: забытый в зале
Кий на бильярде отдыхал,
На смятом канапе лежал
Манежный хлыстик. Таня дале;
Старушка ей: «А вот камин;
Здесь барин сиживал один.


XVIII


Здесь с ним обедывал зимою
Покойный Ленский, наш сосед.
Сюда пожалуйте, за мною.
Вот это барский кабинет;
Здесь почивал он, кофей кушал,
Приказчика доклады слушал
И книжку поутру читал…
И старый барин здесь живал;
Со мной, бывало, в воскресенье,
Здесь под окном, надев очки,
Играть изволил в дурачки.
Дай Бог душе его спасенье,
А косточкам его покой
В могиле, в мать-земле сырой!»


XIX


Татьяна взором умиленным
Вокруг себя на всё глядит,
И всё ей кажется бесценным,
Всё душу томную живит
Полумучительной отрадой:
И стол с померкшею лампадой,
И груда книг, и под окном
Кровать, покрытая ковром,
И вид в окно сквозь сумрак лунный,
И этот бледный полусвет,
И лорда Байрона портрет,
И столбик с куклою чугунной
Под шляпой с пасмурным челом,
С руками, сжатыми крестом.


XX


Татьяна долго в келье модной
Как очарована стоит.
Но поздно. Ветер встал холодный.
Темно в долине. Роща спит
Над отуманенной рекою;
Луна сокрылась за горою,
И пилигримке молодой
Пора, давно пора домой.
И Таня, скрыв свое волненье,
Не без того, чтоб не вздохнуть,
Пускается в обратный путь.
Но прежде просит позволенья
Пустынный замок навещать,
Чтоб книжки здесь одной читать.


XXI


Татьяна с ключницей простилась
За воротами. Через день
Уж утром рано вновь явилась
Она в оставленную сень,
И в молчаливом кабинете,
Забыв на время всё на свете,
Осталась наконец одна,
И долго плакала она.
Потом за книги принялася.
Сперва ей было не до них,
Но показался выбор их
Ей странен. Чтенью предалася
Татьяна жадною душой;
И ей открылся мир иной.


XXII


Хотя мы знаем, что Евгений
Издавна чтенье разлюбил,
Однако ж несколько творений
Он из опалы исключил:
Певца Гяура и Жуана
Да с ним еще два-три романа,
В которых отразился век
И современный человек
Изображен довольно верно
С его безнравственной душой,
Себялюбивой и сухой,
Мечтанью преданной безмерно,
С его озлобленным умом,
Кипящим в действии пустом.


XXIII


Хранили многие страницы
Отметку резкую ногтей;
Глаза внимательной девицы
Устремлены на них живей.
Татьяна видит с трепетаньем,
Какою мыслью, замечаньем
Бывал Онегин поражен,
В чем молча соглашался он.
На их полях она встречает
Черты его карандаша.
Везде Онегина душа
Себя невольно выражает
То кратким словом, то крестом,
То вопросительным крючком.


XXIV


И начинает понемногу
Моя Татьяна понимать
Теперь яснее – слава Богу —
Того, по ком она вздыхать
Осуждена судьбою властной:
Чудак печальный и опасный,
Созданье ада иль небес,
Сей ангел, сей надменный бес,
Что ж он? Ужели подражанье,
Ничтожный призрак, иль еще
Москвич в Гарольдовом плаще,
Чужих причуд истолкованье,
Слов модных полный лексикон?..
Уж не пародия ли он?


XXV


Ужель загадку разрешила?
Ужели слово найдено?
Часы бегут: она забыла,
Что дома ждут ее давно,
Где собралися два соседа
И где об ней идет беседа.
«Как быть? Татьяна не дитя, —
Старушка молвила кряхтя. —
Ведь Оленька ее моложе.
Пристроить девушку, ей-ей,
Пора; а что мне делать с ней?
Всем наотрез одно и то же:
Нейду. И все грустит она
Да бродит по лесам одна».


XXVI


«Не влюблена ль она?» – «В кого же?
Буянов сватался: отказ.
Ивану Петушкову – тоже.
Гусар Пыхтин гостил у нас;
Уж как он Танею прельщался,
Как мелким бесом рассыпался!
Я думала: пойдет авось;
Куда! и снова дело врозь». —
«Что ж, матушка? за чем же стало?
В Москву, на ярманку невест!
Там, слышно, много праздных мест» —
«Ох, мой отец! доходу мало». —
«Довольно для одной зимы,
Не то уж дам хоть я взаймы».


XXVII


Старушка очень полюбила
Совет разумный и благой;
Сочлась – и тут же положила
В Москву отправиться зимой.
И Таня слышит новость эту.
На суд взыскательному свету
Представить ясные черты
Провинциальной простоты,
И запоздалые наряды,
И запоздалый склад речей;
Московских франтов и Цирцей
Привлечь насмешливые взгляды!..
О страх! нет, лучше и верней
В глуши лесов остаться ей.


XXVIII


Вставая с первыми лучами,
Теперь она в поля спешит
И, умиленными очами
Их озирая, говорит:
«Простите, мирные долины,
И вы, знакомых гор вершины,
И вы, знакомые леса;
Прости, небесная краса,
Прости, веселая природа;
Меняю милый, тихий свет
На шум блистательных сует…
Прости ж и ты, моя свобода!
Куда, зачем стремлюся я?
Что мне сулит судьба моя?»


XXIX


Ее прогулки длятся доле.
Теперь то холмик, то ручей
Остановляют поневоле
Татьяну прелестью своей.
Она, как с давними друзьями,
С своими рощами, лугами
Еще беседовать спешит.
Но лето быстрое летит.
Настала осень золотая.
Природа трепетна, бледна,
Как жертва, пышно убрана…
Вот север, тучи нагоняя,
Дохнул, завыл – и вот сама
Идет волшебница зима.


XXX


Пришла, рассыпалась; клоками
Повисла на суках дубов;
Легла волнистыми коврами
Среди полей, вокруг холмов;
Брега с недвижною рекою
Сравняла пухлой пеленою;
Блеснул мороз. И рады мы
Проказам матушки зимы.
Не радо ей лишь сердце Тани.
Нейдет она зиму встречать,
Морозной пылью подышать
И первым снегом с кровли бани
Умыть лицо, плеча и грудь:
Татьяне страшен зимний путь.


XXXI


Отъезда день давно просрочен,
Проходит и последний срок.
Осмотрен, вновь обит, упрочен
Забвенью брошенный возок.
Обоз обычный, три кибитки
Везут домашние пожитки,
Кастрюльки, стулья, сундуки,
Варенье в банках, тюфяки,
Перины, клетки с петухами,
Горшки, тазы et cetera,
Ну, много всякого добра.
И вот в избе между слугами
Поднялся шум, прощальный плач:
Ведут на двор осьмнадцать кляч,


XXXII


В возок боярский их впрягают,
Готовят завтрак повара,
Горой кибитки нагружают,
Бранятся бабы, кучера.
На кляче тощей и косматой
Сидит форейтор бородатый,
Сбежалась челядь у ворот
Прощаться с барами. И вот
Уселись, и возок почтенный,
Скользя, ползет за ворота.
«Простите, мирные места!
Прости, приют уединенный!
Увижу ль вас?..» И слез ручей
У Тани льется из очей.


XXXIII


Когда благому просвещенью
Отдвинем более границ,
Со временем (по расчисленью
Философических таблиц,
Лет чрез пятьсот) дороги, верно,
У нас изменятся безмерно:
Шоссе Россию здесь и тут,
Соединив, пересекут.
Мосты чугунные чрез воды
Шагнут широкою дугой,
Раздвинем горы, под водой
Пророем дерзостные своды,
И заведет крещеный мир
На каждой станции трактир.


XXXIV


Теперь у нас дороги плохи67,
Мосты забытые гниют,
На станциях клопы да блохи
Заснуть минуты не дают;
Трактиров нет. В избе холодной
Высокопарный, но голодный
Для виду прейскурант висит
И тщетный дразнит аппетит,
Меж тем как сельские циклопы
Перед медлительным огнем
Российским лечат молотком
Изделье легкое Европы,
Благословляя колеи
И рвы отеческой земли.


XXXV


Зато зимы порой холодной
Езда приятна и легка.
Как стих без мысли в песне модной
Дорога зимняя гладка.
Автомедоны наши бойки,
Неутомимы наши тройки,
И версты, теша праздный взор,
В глазах мелькают как забор68.
К несчастью, Ларина тащилась,
Боясь прогонов дорогих,
Не на почтовых, на своих,
И наша дева насладилась
Дорожной скукою вполне:
Семь суток ехали оне.


XXXVI


Но вот уж близко. Перед ними
Уж белокаменной Москвы,
Как жар, крестами золотыми
Горят старинные главы.
Ах, братцы! как я был доволен,
Когда церквей и колоколен,
Садов, чертогов полукруг
Открылся предо мною вдруг!
Как часто в горестной разлуке,
В моей блуждающей судьбе,
Москва, я думал о тебе!
Москва… как много в этом звуке
Для сердца русского слилось!
Как много в нем отозвалось!


XXXVII


Вот, окружен своей дубравой,
Петровский замок. Мрачно он
Недавнею гордится славой.
Напрасно ждал Наполеон,
Последним счастьем упоенный,
Москвы коленопреклоненной
С ключами старого Кремля;
Нет, не пошла Москва моя
К нему с повинной головою.
Не праздник, не приемный дар,
Она готовила пожар
Нетерпеливому герою.
Отселе, в думу погружен,
Глядел на грозный пламень он.


XXXVIII


Прощай, свидетель падшей славы,
Петровский замок. Ну! не стой,
Пошел! Уже столпы заставы
Белеют; вот уж по Тверской
Возок несется чрез ухабы.
Мелькают мимо будки, бабы,
Мальчишки, лавки, фонари,
Дворцы, сады, монастыри,
Бухарцы, сани, огороды,
Купцы, лачужки, мужики,
Бульвары, башни, казаки,
Аптеки, магазины моды,
Балконы, львы на воротах
И стаи галок на крестах.


XXXIX. XL


В сей утомительной прогулке
Проходит час-другой, и вот
У Харитонья в переулке
Возок пред домом у ворот
Остановился. К старой тетке,
Четвертый год больной в чахотке,
Они приехали теперь.
Им настежь отворяет дверь,
В очках, в изорванном кафтане,
С чулком в руке, седой калмык.
Встречает их в гостиной крик
Княжны, простертой на диване.
Старушки с плачем обнялись,
И восклицанья полились.


XLI


«Княжна, mon ange!» – «Pachette!»69 – –«–Алина»! —
«Кто б мог подумать? Как давно!
Надолго ль? Милая! Кузина!
Садись – как это мудрено!
Ей-богу, сцена из романа…» —
«А это дочь моя, Татьяна». —
«Ах, Таня! подойди ко мне —
Как будто брежу я во сне…
Кузина, помнишь Грандисона?»
«Как, Грандисон?.. а, Грандисон!
Да, помню, помню. Где же он?» —
«В Москве, живет у Симеона;
Меня в сочельник навестил;
Недавно сына он женил.


XLII


А тот… но после всё расскажем,
Не правда ль? Всей ее родне
Мы Таню завтра же покажем.
Жаль, разъезжать нет мочи мне:
Едва, едва таскаю ноги.
Но вы замучены с дороги;
Пойдемте вместе отдохнуть…
Ох, силы нет… устала грудь…
Мне тяжела теперь и радость,
Не только грусть… душа моя,
Уж никуда не годна я…
Под старость жизнь такая гадость…»
И тут, совсем утомлена,
В слезах раскашлялась она.


XLIII


Больной и ласки и веселье
Татьяну трогают; но ей
Не хорошо на новоселье,
Привыкшей к горнице своей.
Под занавескою шелковой
Не спится ей в постеле новой,
И ранний звон колоколов,
Предтеча утренних трудов,
Ее с постели подымает.
Садится Таня у окна.
Редеет сумрак; но она
Своих полей не различает:
Пред нею незнакомый двор,
Конюшня, кухня и забор.


XLIV


И вот по родственным обедам
Развозят Таню каждый день
Представить бабушкам и дедам
Ее рассеянную лень.
Родне, прибывшей издалеча,
Повсюду ласковая встреча,
И восклицанья, и хлеб-соль.
«Как Таня выросла! Давно ль
Я, кажется, тебя крестила?
А я так на руки брала!
А я так за уши драла!
А я так пряником кормила!»
И хором бабушки твердят:
«Как наши годы-то летят!»


XLV


Но в них не видно перемены;
Всё в них на старый образец:
У тетушки княжны Елены
Всё тот же тюлевый чепец;
Всё белится Лукерья Львовна,
Всё то же лжет Любовь Петровна,
Иван Петрович так же глуп,
Семен Петрович так же скуп,
У Пелагеи Николавны
Всё тот же друг мосье Финмуш,
И тот же шпиц, и тот же муж;
А он, всё клуба член исправный,
Всё так же смирен, так же глух
И так же ест и пьет за двух.


XLVI


Их дочки Таню обнимают.
Младые грации Москвы
Сначала молча озирают
Татьяну с ног до головы;
Ее находят что-то странной,
Провинциальной и жеманной,
И что-то бледной и худой,
А впрочем, очень недурной;
Потом, покорствуя природе,
Дружатся с ней, к себе ведут,
Целуют, нежно руки жмут,
Взбивают кудри ей по моде
И поверяют нараспев
Сердечны тайны, тайны дев.


XLVII


Чужие и свои победы,
Надежды, шалости, мечты.
Текут невинные беседы
С прикрасой легкой клеветы.
Потом, в отплату лепетанья,
Ее сердечного признанья
Умильно требуют оне.
Но Таня, точно как во сне,
Их речи слышит без участья,
Не понимает ничего,
И тайну сердца своего,
Заветный клад и слез и счастья,
Хранит безмолвно между тем
И им не делится ни с кем.


XLVIII


Татьяна вслушаться желает
В беседы, в общий разговор;
Но всех в гостиной занимает
Такой бессвязный, пошлый вздор;
Всё в них так бледно, равнодушно;
Они клевещут даже скучно;
В бесплодной сухости речей,
Расспросов, сплетен и вестей
Не вспыхнет мысли в целы сутки,
Хоть невзначай, хоть наобум
Не улыбнется томный ум,
Не дрогнет сердце, хоть для шутки.
И даже глупости смешной
В тебе не встретишь, свет пустой.


XLIX


Архивны юноши толпою
На Таню чопорно глядят
И про нее между собою
Неблагосклонно говорят.
Один какой-то шут печальный
Ее находит идеальной
И, прислонившись у дверей,
Элегию готовит ей.
У скучной тетки Таню встретя,
К ней как-то Вяземский подсел
И душу ей занять успел.
И, близ него ее заметя,
Об ней, поправя свой парик,
Осведомляется старик.


L


Но там, где Мельпомены бурной
Протяжный раздается вой,
Где машет мантией мишурной
Она пред хладною толпой,
Где Талия тихонько дремлет
И плескам дружеским не внемлет,
Где Терпсихоре лишь одной
Дивится зритель молодой
(Что было также в прежни леты,
Во время ваше и мое),
Не обратились на нее
Ни дам ревнивые лорнеты,
Ни трубки модных знатоков
Из лож и кресельных рядов.


LI


Ее привозят и в Собранье.
Там теснота, волненье, жар,
Музыки грохот, свеч блистанье,
Мельканье, вихорь быстрых пар,
Красавиц легкие уборы,
Людьми пестреющие хоры,
Невест обширный полукруг,
Всё чувства поражает вдруг.
Здесь кажут франты записные
Свое нахальство, свой жилет
И невнимательный лорнет.
Сюда гусары отпускные
Спешат явиться, прогреметь,
Блеснуть, пленить и улететь.


LII


У ночи много звезд прелестных,
Красавиц много на Москве.
Но ярче всех подруг небесных
Луна в воздушной синеве.
Но та, которую не смею
Тревожить лирою моею,
Как величавая луна,
Средь жен и дев блестит одна.
С какою гордостью небесной
Земли касается она!
Как негой грудь ее полна!
Как томен взор ее чудесный!..
Но полно, полно; перестань:
Ты заплатил безумству дань.


LIII


Шум, хохот, беготня, поклоны,
Галоп, мазурка, вальс… Меж тем
Между двух теток, у колонны,
Не замечаема никем,
Татьяна смотрит и не видит,
Волненье света ненавидит;
Ей душно здесь… Она мечтой
Стремится к жизни полевой,
В деревню, к бедным поселянам,
В уединенный уголок,
Где льется светлый ручеек,
К своим цветам, к своим романам
И в сумрак липовых аллей,
Туда, где он являлся ей.


LIV


Так мысль ее далече бродит:
Забыт и свет и шумный бал,
А глаз меж тем с нее не сводит
Какой-то важный генерал.
Друг другу тетушки мигнули,
И локтем Таню враз толкнули,
И каждая шепнула ей:
«Взгляни налево поскорей». —
«Налево? где? что там такое?» —
«Ну, что бы ни было, гляди…
В той кучке, видишь? впереди,
Там, где еще в мундирах двое…
Вот отошел… вот боком стал… —
«Кто? толстый этот генерал?»


LV


Но здесь с победою поздравим
Татьяну милую мою
И в сторону свой путь направим,
Чтоб не забыть, о ком пою…
Да кстати, здесь о том два слова:
Пою приятеля младого 
И множество его причуд. 
Благослови мой долгий труд, 
О ты, эпическая муза! 
И, верный посох мне вручив, 
Не дай блуждать мне вкось и вкрив. 
Довольно. С плеч долой обуза!
Я классицизму отдал честь:
Хоть поздно, а вступленье есть.



Глава восьмая

Fare thee well, and if for ever
Still for ever, fare thee well.
Byron 70



I


В те дни, когда в садах Лицея
Я безмятежно расцветал,
Читал охотно Апулея71,
А Цицерона72не читал,
В те дни в таинственных долинах,
Весной, при кликах лебединых,
Близ вод, сиявших в тишине,
Являться муза стала мне.
Моя студенческая келья
Вдруг озарилась: муза в ней
Открыла пир младых затей,
Воспела детские веселья,
И славу нашей старины,
И сердца трепетные сны.


II


И свет ее с улыбкой встретил;
Успех нас первый окрылил;
Старик Державин нас заметил
И, в гроб сходя, благословил.
……………………………………


III


И я, в закон себе вменяя
Страстей единый произвол,
С толпою чувства разделяя,
Я музу резвую привел
На шум пиров и буйных споров,
Грозы полуночных дозоров;
И к ним в безумные пиры
Она несла свои дары
И как вакханочка резвилась,
За чашей пела для гостей,
И молодежь минувших дней
За нею буйно волочилась,
А я гордился меж друзей
Подругой ветреной моей.


IV


Но я отстал от их союза
И вдаль бежал… Она за мной.
Как часто ласковая муза
Мне услаждала путь немой
Волшебством тайного рассказа!
Как часто по скалам Кавказа
Она Ленорой, при луне,
Со мной скакала на коне!
Как часто по брегам Тавриды
Она меня во мгле ночной
Водила слушать шум морской,
Немолчный шепот Нереиды,
Глубокий, вечный хор валов,
Хвалебный гимн отцу миров.


V


И, позабыв столицы дальной
И блеск и шумные пиры,
В глуши Молдавии печальной
Она смиренные шатры
Племен бродящих посещала,
И между ими одичала,
И позабыла речь богов
Для скудных, странных языков,
Для песен степи, ей любезной…
Вдруг изменилось всё кругом,
И вот она в саду моем
Явилась барышней уездной,
С печальной думою в очах,
С французской книжкою в руках.


VI


И ныне музу я впервые
На светский раут73привожу;
На прелести ее степные
С ревнивой робостью гляжу.
Сквозь тесный ряд аристократов,
Военных франтов, дипломатов
И гордых дам она скользит;
Вот села тихо и глядит,
Любуясь шумной теснотою,
Мельканьем платьев и речей,
Явленьем медленным гостей
Перед хозяйкой молодою,
И темной рамою мужчин
Вкруг дам, как около картин.


VII


Ей нравится порядок стройный
Олигархических бесед,
И холод гордости спокойной,
И эта смесь чинов и лет.
Но это кто в толпе избранной
Стоит безмолвный и туманный?
Для всех он кажется чужим.
Мелькают лица перед ним,
Как ряд докучных привидений.
Что, сплин иль страждущая спесь
В его лице? Зачем он здесь?
Кто он таков? Ужель Евгений?
Ужели он?.. Так, точно он.
– Давно ли к нам он занесен?


VIII


Всё тот же ль он иль усмирился?
Иль корчит так же чудака?
Скажите, чем он возвратился?
Что нам представит он пока?
Чем ныне явится? Мельмотом,
Космополитом, патриотом,
Гарольдом, квакером, ханжой,
Иль маской щегольнет иной,
Иль просто будет добрый малой,
Как вы да я, как целый свет?
По крайней мере мой совет:
Отстать от моды обветшалой.
Довольно он морочил свет…
– Знаком он вам? – И да и нет.


IX


– Зачем же так неблагосклонно
Вы отзываетесь о нем?
За то ль, что мы неугомонно
Хлопочем, судим обо всем,
Что пылких душ неосторожность
Самолюбивую ничтожность
Иль оскорбляет, иль смешит,
Что ум, любя простор, теснит,
Что слишком часто разговоры
Принять мы рады за дела,
Что глупость ветрена и зла,
Что важным людям важны вздоры,
И что посредственность одна
Нам по плечу и не странна?


X


Блажен, кто смолоду был молод,
Блажен, кто вовремя созрел,
Кто постепенно жизни холод
С летами вытерпеть умел;
Кто странным снам не предавался,
Кто черни светской не чуждался,
Кто в двадцать лет был франт иль хват,
А в тридцать выгодно женат;
Кто в пятьдесят освободился
От частных и других долгов,
Кто славы, денег и чинов
Спокойно в очередь добился,
О ком твердили целый век:
N. N. прекрасный человек.


XI


Но грустно думать, что напрасно
Была нам молодость дана,
Что изменяли ей всечасно,
Что обманула нас она;
Что наши лучшие желанья,
Что наши свежие мечтанья
Истлели быстрой чередой,
Как листья осенью гнилой.
Несносно видеть пред собою
Одних обедов длинный ряд,
Глядеть на жизнь как на обряд
И вслед за чинною толпою
Идти, не разделяя с ней
Ни общих мнений, ни страстей.


XII


Предметом став суждений шумных,
Несносно (согласитесь в том)
Между людей благоразумных
Прослыть притворным чудаком,
Или печальным сумасбродом,
Иль сатаническим уродом,
Иль даже демоном моим.
Онегин (вновь займуся им),
Убив на поединке друга,
Дожив без цели, без трудов
До двадцати шести годов,
Томясь в бездействии досуга
Без службы, без жены, без дел,
Ничем заняться не умел.


XIII


Им овладело беспокойство,
Охота к перемене мест
(Весьма мучительное свойство,
Немногих добровольный крест).
Оставил он свое селенье,
Лесов и нив уединенье,
Где окровавленная тень
Ему являлась каждый день,
И начал странствия без цели,
Доступный чувству одному;
И путешествия ему,
Как всё на свете, надоели;
Он возвратился и попал,
Как Чацкий, с корабля на бал.


XIV


Но вот толпа заколебалась,
По зале шепот пробежал…
К хозяйке дама приближалась,
За нею важный генерал.
Она была нетороплива,
Не холодна, не говорлива,
Без взора наглого для всех,
Без притязаний на успех,
Без этих маленьких ужимок,
Без подражательных затей…
Всё тихо, просто было в ней,
Она казалась верный снимок
Du comme il faut… (Шишков, прости:
Не знаю, как перевести.)


XV


К ней дамы подвигались ближе;
Старушки улыбались ей;
Мужчины кланялися ниже,
Ловили взор ее очей;
Девицы проходили тише
Пред ней по зале; и всех выше
И нос и плечи подымал
Вошедший с нею генерал.
Никто б не мог ее прекрасной
Назвать; но с головы до ног
Никто бы в ней найти не мог
Того, что модой самовластной
В высоком лондонском кругу
Зовется vulgar 74. (Не могу…


XVI


Люблю я очень это слово,
Но не могу перевести;
Оно у нас покамест ново,
И вряд ли быть ему в чести.
Оно б годилось в эпиграмме…)
Но обращаюсь к нашей даме.
Беспечной прелестью мила,
Она сидела у стола
С блестящей Ниной Воронскою,
Сей Клеопатрою Невы;
И верно б согласились вы,
Что Нина мраморной красою
Затмить соседку не могла,
Хоть ослепительна была.


XVII


«Ужели, – думает Евгений, —
Ужель она? Но точно… Нет…
Как! из глуши степных селений…»
И неотвязчивый лорнет
Он обращает поминутно
На ту, чей вид напомнил смутно
Ему забытые черты.
«Скажи мне, князь, не знаешь ты,
Кто там в малиновом берете
С послом испанским говорит?»
Князь на Онегина глядит.
«Ага! давно ж ты не был в свете.
Постой, тебя представлю я». —
«Да кто ж она?» – «Жена моя».


XVIII


«Так ты женат! не знал я ране!
Давно ли?» – «Около двух лет». —
«На ком?» – «На Лариной». – «Татьяне!»
«Ты ей знаком?» – «Я им сосед». —
«О, так пойдем же». Князь подходит
К своей жене и ей подводит
Родню и друга своего.
Княгиня смотрит на него…
И что ей душу ни смутило,
Как сильно ни была она
Удивлена, поражена,
Но ей ничто не изменило:
В ней сохранился тот же тон,
Был так же тих ее поклон.


XIX


Ей-ей! не то, чтоб содрогнулась
Иль стала вдруг бледна, красна…
У ней и бровь не шевельнулась;
Не сжала даже губ она.
Хоть он глядел нельзя прилежней,
Но и следов Татьяны прежней
Не мог Онегин обрести.
С ней речь хотел он завести
И – и не мог. Она спросила,
Давно ль он здесь, откуда он
И не из их ли уж сторон?
Потом к супругу обратила
Усталый взгляд; скользнула вон…
И недвижим остался он.


XX


Ужель та самая Татьяна,
Которой он наедине,
В начале нашего романа,
В глухой, далекой стороне,
В благом пылу нравоученья
Читал когда-то наставленья,
Та, от которой он хранит
Письмо, где сердце говорит,
Где всё наруже, всё на воле,
Та девочка… иль это сон?..
Та девочка, которой он
Пренебрегал в смиренной доле,
Ужели с ним сейчас была
Так равнодушна, так смела?


XXI


Он оставляет раут тесный,
Домой задумчив едет он;
Мечтой то грустной, то прелестной
Его встревожен поздний сон.
Проснулся он; ему приносят
Письмо: князь N покорно просит
Его на вечер. «Боже! к ней!..
О, буду, буду!» и скорей
Марает он ответ учтивый.
Что с ним? в каком он странном сне!
Что шевельнулось в глубине
Души холодной и ленивой?
Досада? суетность? иль вновь
Забота юности – любовь?


XXII


Онегин вновь часы считает,
Вновь не дождется дню конца.
Но десять бьет; он выезжает,
Он полетел, он у крыльца,
Он с трепетом к княгине входит;
Татьяну он одну находит,
И вместе несколько минут
Они сидят. Слова нейдут
Из уст Онегина. Угрюмый,
Неловкий, он едва-едва
Ей отвечает. Голова
Его полна упрямой думой.
Упрямо смотрит он: она
Сидит покойна и вольна.


XXIII


Приходит муж. Он прерывает
Сей неприятный te^te-а-te^te75;
С Онегиным он вспоминает
Проказы, шутки прежних лет.
Они смеются. Входят гости.
Вот крупной солью светской злости
Стал оживляться разговор;
Перед хозяйкой легкий вздор
Сверкал без глупого жеманства,
И прерывал его меж тем
Разумный толк без пошлых тем,
Без вечных истин, без педантства,
И не пугал ничьих ушей
Свободной живостью своей.


XXIV


Тут был, однако, цвет столицы,
И знать, и моды образцы,
Везде встречаемые лица,
Необходимые глупцы;
Тут были дамы пожилые
В чепцах и в розах, с виду злые;
Тут было несколько девиц,
Не улыбающихся лиц;
Тут был посланник, говоривший
О государственных делах;
Тут был в душистых сединах
Старик, по-старому шутивший:
Отменно тонко и умно,
Что нынче несколько смешно.


XXV


Тут был на эпиграммы падкий,
На всё сердитый господин:
На чай хозяйский слишком сладкий,
На плоскость дам, на тон мужчин,
На толки про роман туманный,
На вензель, двум сестрицам данный,
На ложь журналов, на войну,
На снег и на свою жену.
……………………………………


XXVI


Тут был Проласов, заслуживший
Известность низостью души,
Во всех альбомах притупивший,
St.-Priest, твои карандаши;
В дверях другой диктатор бальный
Стоял картинкою журнальной,
Румян, как вербный херувим,
Затянут, нем и недвижим,
И путешественник залётный,
Перекрахмаленный нахал,
В гостях улыбку возбуждал
Своей осанкою заботной,
И молча обмененный взор
Ему был общий приговор.


XXVII


Но мой Онегин вечер целой
Татьяной занят был одной,
Не этой девочкой несмелой,
Влюбленной, бедной и простой,
Но равнодушною княгиней,
Но неприступною богиней
Роскошной, царственной Невы.
О люди! все похожи вы
На прародительницу Эву:
Что вам дано, то не влечет;
Вас непрестанно змий зовет
К себе, к таинственному древу;
Запретный плод вам подавай,
А без того вам рай не рай.


XXVIII


Как изменилася Татьяна!
Как твердо в роль свою вошла!
Как утеснительного сана
Приемы скоро приняла!
Кто б смел искать девчонки нежной
В сей величавой, в сей небрежной
Законодательнице зал?
И он ей сердце волновал!
Об нем она во мраке ночи,
Пока Морфей не прилетит,
Бывало, девственно грустит,
К луне подъемлет томны очи,
Мечтая с ним когда-нибудь
Свершить смиренный жизни путь!


XXIX


Любви все возрасты покорны;
Но юным, девственным сердцам
Ее порывы благотворны,
Как бури вешние полям:
В дожде страстей они свежеют,
И обновляются, и зреют —
И жизнь могущая дает
И пышный цвет, и сладкий плод.
Но в возраст поздний и бесплодный,
На повороте наших лет,
Печален страсти мертвой след:
Так бури осени холодной
В болото обращают луг
И обнажают лес вокруг.


XXX


Сомненья нет: увы! Евгений
В Татьяну, как дитя, влюблен;
В тоске любовных помышлений
И день и ночь проводит он.
Ума не внемля строгим пеням,
К ее крыльцу, стеклянным сеням
Он подъезжает каждый день;
За ней он гонится, как тень;
Он счастлив, если ей накинет
Боа пушистый на плечо,
Или коснется горячо
Ее руки, или раздвинет
Пред нею пестрый полк ливрей,
Или платок подымет ей.


XXXI


Она его не замечает,
Как он ни бейся, хоть умри.
Свободно дома принимает,
В гостях с ним молвит слова три,
Порой одним поклоном встретит,
Порою вовсе не заметит;
Кокетства в ней ни капли нет —
Его не терпит высший свет.
Бледнеть Онегин начинает:
Ей иль не видно, иль не жаль;
Онегин сохнет, и едва ль
Уж не чахоткою страдает.
Все шлют Онегина к врачам,
Те хором шлют его к водам. 


XXXII


А он не едет; он заране
Писать ко прадедам готов
О скорой встрече; а Татьяне
И дела нет (их пол таков);
А он упрям, отстать не хочет,
Еще надеется, хлопочет;
Смелей здорового, больной
Княгине слабою рукой
Он пишет страстное посланье.
Хоть толку мало вообще
Он в письмах видел не вотще;
Но, знать, сердечное страданье
Уже пришло ему невмочь.
Вот вам письмо его точь-в-точь.


Письмо Онегина к Татьяне

Предвижу всё: вас оскорбит
Печальной тайны объясненье.
Какое горькое презренье
Ваш гордый взгляд изобразит!
Чего хочу? с какою целью
Открою душу вам свою?
Какому злобному веселью,
Быть может, повод подаю!

Случайно вас когда-то встретя,
В вас искру нежности заметя,
Я ей поверить не посмел:
Привычке милой не дал ходу;
Свою постылую свободу
Я потерять не захотел.
Еще одно нас разлучило…
Несчастной жертвой Ленский пал…
Ото всего, что сердцу мило,
Тогда я сердце оторвал;
Чужой для всех, ничем не связан,
Я думал: вольность и покой
Замена счастью. Боже мой!
Как я ошибся, как наказан…

Нет, поминутно видеть вас,
Повсюду следовать за вами,
Улыбку уст, движенье глаз
Ловить влюбленными глазами,
Внимать вам долго, понимать
Душой все ваше совершенство,
Пред вами в муках замирать,
Бледнеть и гаснуть… вот блаженство!

И я лишен того: для вас
Тащусь повсюду наудачу;
Мне дорог день, мне дорог час:
А я в напрасной скуке трачу
Судьбой отсчитанные дни.
И так уж тягостны они.
Я знаю: век уж мой измерен;
Но чтоб продлилась жизнь моя,
Я утром должен быть уверен,
Что с вами днем увижусь я…

Боюсь, в мольбе моей смиренной
Увидит ваш суровый взор
Затеи хитрости презренной —
И слышу гневный ваш укор.
Когда б вы знали, как ужасно
Томиться жаждою любви,
Пылать – и разумом всечасно
Смирять волнение в крови;
Желать обнять у вас колени
И, зарыдав, у ваших ног
Излить мольбы, признанья, пени,
Всё, всё, что выразить бы мог,
А между тем притворным хладом
Вооружать и речь и взор,
Вести спокойный разговор,
Глядеть на вас веселым взглядом!..
Но так и быть: я сам себе
Противиться не в силах боле;
Всё решено: я в вашей воле,
И предаюсь моей судьбе.


XXXIII


Ответа нет. Он вновь посланье:
Второму, третьему письму
Ответа нет. В одно собранье
Он едет; лишь вошел… ему
Она навстречу. Как сурова!
Его не видят, с ним ни слова;
У! как теперь окружена
Крещенским холодом она!
Как удержать негодованье
Уста упрямые хотят!
Вперил Онегин зоркий взгляд:
Где, где смятенье, состраданье?
Где пятна слез?.. Их нет, их нет!
На сем лице лишь гнева след…


XXXIV


Да, может быть, боязни тайной,
Чтоб муж иль свет не угадал
Проказы, слабости случайной…
Всего, что мой Онегин знал…
Надежды нет! Он уезжает,
Свое безумство проклинает —
И, в нем глубоко погружен,
От света вновь отрекся он.
И в молчаливом кабинете
Ему припомнилась пора,
Когда жестокая хандра
За ним гналася в шумном свете,
Поймала, за ворот взяла
И в темный угол заперла.


XXXV


Стал вновь читать он без разбора.
Прочел он Гиббона, Руссо,
Манзони, Гердера, Шамфора,
Madame de Staёl, Биша, Тиссо,
Прочел скептического Беля,
Прочел творенья Фонтенеля,
Прочел из наших кой-кого,
Не отвергая ничего:
И альманахи, и журналы,
Где поученья нам твердят,
Где нынче так меня бранят,
А где такие мадригалы
Себе встречал я иногда:
Е sempre bene 76, господа.


XXXVI


И что ж? Глаза его читали,
Но мысли были далеко;
Мечты, желания, печали
Теснились в душу глубоко.
Он меж печатными строками
Читал духовными глазами
Другие строки. В них-то он
Был совершенно углублен.
То были тайные преданья
Сердечной, темной старины,
Ни с чем не связанные сны,
Угрозы, толки, предсказанья,
Иль длинной сказки вздор живой,
Иль письма девы молодой.


XXXVII


И постепенно в усыпленье
И чувств и дум впадает он,
А перед ним воображенье
Свой пестрый мечет фараон.
То видит он: на талом снеге,
Как будто спящий на ночлеге,
Недвижим юноша лежит,
И слышит голос: что ж? убит.
То видит он врагов забвенных,
Клеветников и трусов злых,
И рой изменниц молодых,
И круг товарищей презренных,
То сельский дом – и у окна
Сидит она… и всё она!..


XXXVIII


Он так привык теряться в этом,
Что чуть с ума не своротил
Или не сделался поэтом.
Признаться: то-то б одолжил!
А точно: силой магнетизма
Стихов российских механизма
Едва в то время не постиг
Мой бестолковый ученик.
Как походил он на поэта,
Когда в углу сидел один,
И перед ним пылал камин,
И он мурлыкал: Benedetta 77
Иль Idol mio 78и ронял
В огонь то туфлю, то журнал.


XXXIX


Дни мчались: в воздухе нагретом
Уж разрешалася зима;
И он не сделался поэтом,
Не умер, не сошел с ума.
Весна живит его: впервые
Свои покои запертые,
Где зимовал он, как сурок,
Двойные окна, камелек
Он ясным утром оставляет,
Несется вдоль Невы в санях.
На синих, иссеченных льдах
Играет солнце; грязно тает
На улицах разрытый снег.
Куда по нем свой быстрый бег


XL


Стремит Онегин? Вы заране
Уж угадали; точно так:
Примчался к ней, к своей Татьяне,
Мой неисправленный чудак.
Идет, на мертвеца похожий.
Нет ни одной души в прихожей.
Он в залу; дальше: никого.
Дверь отворил он. Что ж его
С такою силой поражает?
Княгиня перед ним, одна,
Сидит, не убрана, бледна,
Письмо какое-то читает
И тихо слезы льет рекой,
Опершись на руку щекой.


XLI


О, кто б немых ее страданий
В сей быстрый миг не прочитал!
Кто прежней Тани, бедной Тани
Теперь в княгине б не узнал!
В тоске безумных сожалений
К ее ногам упал Евгений;
Она вздрогнула и молчит
И на Онегина глядит
Без удивления, без гнева…
Его больной, угасший взор,
Молящий вид, немой укор,
Ей внятно всё. Простая дева,
С мечтами, сердцем прежних дней,
Теперь опять воскресла в ней.


XLII


Она его не подымает
И, не сводя с него очей,
От жадных уст не отымает
Бесчувственной руки своей…
О чем теперь ее мечтанье?
Проходит долгое молчанье,
И тихо наконец она:
«Довольно; встаньте. Я должна
Вам объясниться откровенно.
Онегин, помните ль тот час,
Когда в саду, в аллее нас
Судьба свела, и так смиренно
Урок ваш выслушала я?
Сегодня очередь моя.


XLIII


«Онегин, я тогда моложе,
Я лучше, кажется, была,
И я любила вас; и что же?
Что в сердце вашем я нашла?
Какой ответ? одну суровость.
Не правда ль? Вам была не новость
Смиренной девочки любовь?
И нынче – Боже! – стынет кровь,
Как только вспомню взгляд холодный
И эту проповедь… Но вас
Я не виню: в тот страшный час
Вы поступили благородно,
Вы были правы предо мной.
Я благодарна всей душой…


XLIV


Тогда – не правда ли? – в пустыне,
Вдали от суетной молвы,
Я вам не нравилась… Что ж ныне
Меня преследуете вы?
Зачем у вас я на примете?
Не потому ль, что в высшем свете
Теперь являться я должна;
Что я богата и знатна,
Что муж в сраженьях изувечен,
Что нас за то ласкает двор?
Не потому ль, что мой позор
Теперь бы всеми был замечен
И мог бы в обществе принесть
Вам соблазнительную честь?


XLV


Я плачу… если вашей Тани
Вы не забыли до сих пор,
То знайте: колкость вашей брани,
Холодный, строгий разговор,
Когда б в моей лишь было власти,
Я предпочла б обидной страсти
И этим письмам и слезам.
К моим младенческим мечтам
Тогда имели вы хоть жалость,
Хоть уважение к летам…
А нынче! – что к моим ногам
Вас привело? какая малость!
Как с вашим сердцем и умом
Быть чувства мелкого рабом?


XLVI


А мне, Онегин, пышность эта,
Постылой жизни мишура,
Мои успехи в вихре света,
Мой модный дом и вечера,
Что в них? Сейчас отдать я рада
Всю эту ветошь маскарада,
Весь этот блеск, и шум, и чад
За полку книг, за дикий сад,
За наше бедное жилище,
За те места, где в первый раз,
Онегин, видела я вас,
Да за смиренное кладбище,
Где нынче крест и тень ветвей
Над бедной нянею моей…


XLVII


А счастье было так возможно,
Так близко!.. Но судьба моя
Уж решена. Неосторожно,
Быть может, поступила я:
Меня с слезами заклинаний
Молила мать; для бедной Тани
Все были жребии равны…
Я вышла замуж. Вы должны,
Я вас прошу, меня оставить;
Я знаю: в вашем сердце есть
И гордость, и прямая честь.
Я вас люблю (к чему лукавить?),
Но я другому отдана;
Я буду век ему верна».


XLVIII


Она ушла. Стоит Евгений,
Как будто громом поражен.
В какую бурю ощущений
Теперь он сердцем погружен!
Но шпор незапный звон раздался,
И муж Татьянин показался,
И здесь героя моего,
В минуту, злую для него,
Читатель, мы теперь оставим,
Надолго… навсегда. За ним
Довольно мы путем одним
Бродили по свету. Поздравим
Друг друга с берегом. Ура!
Давно б (не правда ли?) пора!


XLIX


Кто б ни был ты, о мой читатель,
Друг, недруг, я хочу с тобой
Расстаться нынче как приятель.
Прости. Чего бы ты за мной
Здесь ни искал в строфах небрежных,
Воспоминаний ли мятежных,
Отдохновенья ль от трудов,
Живых картин, иль острых слов,
Иль грамматических ошибок,
Дай Бог, чтоб в этой книжке ты
Для развлеченья, для мечты,
Для сердца, для журнальных сшибок
Хотя крупицу мог найти.
За сим расстанемся, прости!


L


Прости ж и ты, мой спутник странный,
И ты, мой верный идеал,
И ты, живой и постоянный,
Хоть малый труд. Я с вами знал
Всё, что завидно для поэта:
Забвенье жизни в бурях света,
Беседу сладкую друзей.
Промчалось много, много дней
С тех пор, как юная Татьяна
И с ней Онегин в смутном сне
Явилися впервые мне —
И даль свободного романа
Я сквозь магический кристалл
Еще не ясно различал.


LI


Но те, которым в дружной встрече
Я строфы первые читал…
Иных уж нет, а те далече,
Как Сади некогда сказал.
Без них Онегин дорисован.
А та, с которой образован
Татьяны милый идеал…
О много, много рок отъял!
Блажен, кто праздник жизни рано
Оставил, не допив до дна
Бокала полного вина,
Кто не дочел ее романа
И вдруг умел расстаться с ним,
Как я с Онегиным моим.


Конец 


Отрывки из путешествия Онегина

Последняя глава «Евгения Онегина» издана была особо, с следующим предисловием:
«Пропущенные строфы подавали неоднократно повод к порицанию и насмешкам (впрочем, весьма справедливым и остроумным). Автор чистосердечно признается, что он выпустил из своего романа целую главу, в коей описано было путешествие Онегина по России. От него зависело означить сию выпущенную главу точками или цифром; но во избежание соблазна решился он лучше выставить вместо девятого нумера осьмой над последней главою Евгения Онегина и пожертвовать одною из окончательных строф:

Пора: перо покоя просит;
Я девять песен написал;
На берег радостный выносит
Мою ладью девятый вал —
Хвала вам, девяти каменам, и проч.».

П.А. Катенин (коему прекрасный поэтический талант не мешает быть и тонким критиком) заметил нам, что сие исключение, может быть и выгодное для читателей, вредит, однако ж, плану целого сочинения; ибо чрез то переход от Татьяны, уездной барышни, к Татьяне, знатной даме, становится слишком неожиданным и необъясненным. – Замечание, обличающее опытного художника. Автор сам чувствовал справедливость оного, но решился выпустить эту главу по причинам, важным для него, а не для публики. Некоторые отрывки были напечатаны; мы здесь их помещаем, присовокупив к ним еще несколько строф.
Е. Онегин из Москвы едет в Нижний Новгород:

……… перед ним
Макарьев суетно хлопочет,
Кипит обилием своим.
Сюда жемчуг привез индеец,
Поддельны вины европеец,
Табун бракованых коней
Пригнал заводчик из степей,
Игрок привез свои колоды
И горсть услужливых костей,
Помещик – спелых дочерей,
А дочки – прошлогодни моды.
Всяк суетится, лжет за двух,
И всюду меркантильный дух.


* * *

Тоска!..
Онегин едет в Астрахань и оттуда на Кавказ.

Он видит: Терек своенравный
Крутые роет берега;
Пред ним парит орел державный,
Стоит олень, склонив рога;
Верблюд лежит в тени утеса,
В лугах несется конь черкеса,
И вкруг кочующих шатров
Пасутся овцы калмыков,
Вдали – кавказские громады:
К ним путь открыт. Пробилась брань
За их естественную грань,
Чрез их опасные преграды;
Брега Арагвы и Куры
Узрели русские шатры.


* * *

Уже пустыни сторож вечный,
Стесненный холмами вокруг,
Стоит Бешту остроконечный
И зеленеющий Машук,
Машук, податель струй целебных;
Вокруг ручьев его волшебных
Больных теснится бледный рой;
Кто жертва чести боевой,
Кто почечуя, кто Киприды;
Страдалец мыслит жизни нить
В волнах чудесных укрепить,
Кокетка злых годов обиды
На дне оставить, а старик
Помолодеть – хотя на миг.


* * *

Питая горьки размышленья,
Среди печальной их семьи,
Онегин взором сожаленья
Глядит на дымные струи
И мыслит, грустью отуманен:
Зачем я пулей в грудь не ранен?
Зачем не хилый я старик,
Как этот бедный откупщик?
Зачем, как тульский заседатель,
Я не лежу в параличе?
Зачем не чувствую в плече
Хоть ревматизма? – ах, Создатель!
Я молод, жизнь во мне крепка;
Чего мне ждать? тоска, тоска!..

Онегин посещает потом Тавриду:

Воображенью край священный:
С Атридом спорил там Пилад,
Там закололся Митридат,
Там пел Мицкевич вдохновенный
И, посреди прибрежных скал,
Свою Литву воспоминал.


* * *

Прекрасны вы, брега Тавриды,
Когда вас видишь с корабля
При свете утренней Киприды,
Как вас впервой увидел я;
Вы мне предстали в блеске брачном:
На небе синем и прозрачном
Сияли груды ваших гор,
Долин, деревьев, сёл узор
Разостлан был передо мною.
А там, меж хижинок татар…
Какой во мне проснулся жар!
Какой волшебною тоскою
Стеснялась пламенная грудь!
Но, муза! прошлое забудь.


* * *

Какие б чувства ни таились
Тогда во мне – теперь их нет:
Они прошли иль изменились…
Мир вам, тревоги прошлых лет!
В ту пору мне казались нужны
Пустыни, волн края жемчужны,
И моря шум, и груды скал,
И гордой девы идеал,
И безыменные страданья…
Другие дни, другие сны;
Смирились вы, моей весны
Высокопарные мечтанья,
И в поэтический бокал
Воды я много подмешал.


* * *

Иные нужны мне картины:
Люблю песчаный косогор,
Перед избушкой две рябины,
Калитку, сломанный забор,
На небе серенькие тучи,
Перед гумном соломы кучи
Да пруд под сенью ив густых,
Раздолье уток молодых;
Теперь мила мне балалайка
Да пьяный топот трепака
Перед порогом кабака.
Мой идеал теперь – хозяйка,
Мои желания – покой,
Да щей горшок, да сам большой. 


* * *

Порой дождливою намедни
Я, завернув на скотный двор…
Тьфу! прозаические бредни,
Фламандской школы пестрый сор!
Таков ли был я, расцветая?
Скажи, фонтан Бахчисарая!
Такие ль мысли мне на ум
Навел твой бесконечный шум,
Когда безмолвно пред тобою
Зарему я воображал
Средь пышных, опустелых зал…
Спустя три года, вслед за мною,
Скитаясь в той же стороне,
Онегин вспомнил обо мне.


* * *

Я жил тогда в Одессе пыльной…
Там долго ясны небеса,
Там хлопотливо торг обильный
Свои подъемлет паруса;
Там всё Европой дышит, веет,
Всё блещет югом и пестреет
Разнообразностью живой.
Язык Италии златой
Звучит по улице веселой,
Где ходит гордый славянин,
Француз, испанец, армянин,
И грек, и молдаван тяжелый,
И сын египетской земли,
Корсар в отставке, Морали.


* * *

Одессу звучными стихами
Наш друг Туманский описал,
Но он пристрастными глазами
В то время на нее взирал.
Приехав, он прямым поэтом
Пошел бродить с своим лорнетом
Один над морем – и потом
Очаровательным пером
Сады одесские прославил.
Всё хорошо, но дело в том,
Что степь нагая там кругом;
Кой-где недавный труд заставил
Младые ветви в знойный день
Давать насильственную тень.


* * *

А где, бишь, мой рассказ несвязный?
В Одессе пыльной, я сказал.
Я б мог сказать: в Одессе грязной —
И тут бы, право, не солгал.
В году недель пять-шесть Одесса,
По воле бурного Зевеса,
Потоплена, запружена,
В густой грязи погружена.
Все домы на аршин загрязнут,
Лишь на ходулях пешеход
По улице дерзает вброд;
Кареты, люди тонут, вязнут,
И в дрожках вол, рога склоня,
Сменяет хилого коня.


* * *

Но уж дробит каменья молот,
И скоро звонкой мостовой
Покроется спасенный город,
Как будто кованой броней.
Однако в сей Одессе влажной
Еще есть недостаток важный;
Чего б вы думали? – воды.
Потребны тяжкие труды…
Что ж? это небольшое горе,
Особенно, когда вино
Без пошлины привезено.
Но солнце южное, но море…
Чего ж вам более, друзья?
Благословенные края!


* * *

Бывало, пушка зоревая
Лишь только грянет с корабля,
С крутого берега сбегая,
Уж к морю отправляюсь я.
Потом за трубкой раскаленной,
Волной соленой оживленный,
Как мусульман в своем раю,
С восточной гущей кофе пью.
Иду гулять. Уж благосклонный
Открыт Casino; чашек звон
Там раздается; на балкон
Маркёр выходит полусонный
С метлой в руках, и у крыльца
Уже сошлися два купца.


* * *

Глядишь – и площадь запестрела.
Всё оживилось; здесь и там
Бегут за делом и без дела,
Однако больше по делам.
Дитя расчета и отваги,
Идет купец взглянуть на флаги,
Проведать, шлют ли небеса
Ему знакомы паруса.
Какие новые товары
Вступили нынче в карантин?
Пришли ли бочки жданных вин?
И что чума? и где пожары?
И нет ли голода, войны
Или подобной новизны?


* * *

Но мы, ребята без печали,
Среди заботливых купцов,
Мы только устриц ожидали
От цареградских берегов.
Что устрицы? пришли! О радость!
Летит обжорливая младость
Глотать из раковин морских
Затворниц жирных и живых,
Слегка обрызнутых лимоном.
Шум, споры – легкое вино
Из погребов принесено
На стол услужливым Отоном79;
Часы летят, а грозный счет
Меж тем невидимо растет.


* * *

Но уж темнеет вечер синий,
Пора нам в оперу скорей:
Там упоительный Россини,
Европы баловень – Орфей.
Не внемля критике суровой,
Он вечно тот же, вечно новый,
Он звуки льет – они кипят,
Они текут, они горят,
Как поцелуи молодые,
Все в неге, в пламени любви,
Как зашипевшего аи
Струя и брызги золотые…
Но, господа, позволено ль
С вином равнять do-re-mi-sol?


* * *

А только ль там очарований?
А разыскательный лорнет?
А закулисные свиданья?
A prima donna? а балет?
А ложа, где, красой блистая,
Негоциантка молодая,
Самолюбива и томна,
Толпой рабов окружена?
Она и внемлет и не внемлет
И каватине, и мольбам,
И шутке с лестью пополам…
А муж – в углу за нею дремлет,
Впросонках фора закричит,
Зевнет и – снова захрапит.


* * *

Финал гремит; пустеет зала;
Шумя, торопится разъезд;
Толпа на площадь побежала
При блеске фонарей и звезд,
Сыны Авзонии счастливой
Слегка поют мотив игривый,
Его невольно затвердив,
А мы ревем речитатив.
Но поздно. Тихо спит Одесса;
И бездыханна и тепла
Немая ночь. Луна взошла,
Прозрачно-легкая завеса
Объемлет небо. Всё молчит;
Лишь море Черное шумит…


* * *

Итак, я жил тогда в Одессе…

"""